# Importação de Bibliotecas

In [ ]:
#  Importação de bibliotecas completas para experimentos com MLflow e DVC/Dagshub

# --- 1. Manipulação de Dados e Sistema ---
import numpy as np
import pandas as pd
import os
import io
import warnings
import tempfile
from tempfile import mkdtemp
from dotenv import load_dotenv, find_dotenv

# --- 2. Visualização ---
import matplotlib.pyplot as plt
import seaborn as sns

# --- 3. Versionamento e Conexão (DVC/Dagshub) ---
import dvc.api
from dagshub.data_engine import datasources
import dagshub

# --- 4. Rastreamento de Experimentos (MLflow) ---
import mlflow
import mlflow.sklearn
import mlflow.catboost
import mlflow.lightgbm
import mlflow.xgboost
import mlflow.models.signature
from mlflow.models import infer_signature
from mlflow.tracking import MlflowClient

# --- 5. Machine Learning (Scikit-learn) ---
# 5.1. Modelos
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, StackingClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

# 5.2. Pré-processamento
from sklearn.preprocessing import RobustScaler, StandardScaler, LabelEncoder

# 5.3. Seleção e Validação de Modelos
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, StratifiedKFold

# 5.4. Métricas de Avaliação
from sklearn.metrics import (
    accuracy_score,
    recall_score,
    f1_score,
    precision_recall_fscore_support,
    classification_report,
    confusion_matrix,
    make_scorer,
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    mean_absolute_percentage_error
)
# Alias para skm, se preferir usar (opcional)
import sklearn.metrics as skm

# --- 6. Bibliotecas de Gradient Boosting ---
import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor
from xgboost.callback import EarlyStopping as XGBCallback  # Renomeado para evitar conflito
import lightgbm as lgb
from lightgbm import LGBMClassifier
import catboost
from catboost import CatBoostClassifier, CatBoostRegressor

# --- 7. Utilidades Diversas ---
import requests

# --- Configurações Finais ---

print("Todas as bibliotecas carregadas com sucesso!")

 Todas as bibliotecas carregadas com sucesso!


# Carregamento do Dataset Processado

Nesta etapa, vamos acessar o dataset processado diretamente do repositório versionado no **Dagshub/DVC**.  


In [5]:
# Carrega o dataset processado via Dagshub Data Engine
ds = datasources.get('estrellacouto05/quantum-finance-credit-score', 'processed')

# Exibe as primeiras linhas para verificação
ds.all().dataframe



Accessing as estrellacouto05

Output()

,path,datapoint_id,dagshub_download_url,media type,size
0,credit_score_processed.csv,103598542,https://dagshub.com/api/v1/repos/estrellacouto...,text/plain,32908761


Após obter o objeto `datasource`, agora vamos recuperar o link de download do dataset versionado.   
Com essa URL, carregamos os dados diretamente em um **DataFrame Pandas**.

In [6]:
# Obtém os primeiros registros do datasource e extrai o URL de download
res = ds.head()

for dp in res:
    dataset_url = dp.download_url
    print("Dataset URL:", dataset_url)

# Carrega variáveis de ambiente do arquivo .env
load_dotenv(find_dotenv())

dagshub_token = os.getenv("dagshub_token")
# Cria o cabeçalho de autenticação para a requisição
headers = {
    "Authorization": f"Bearer {dagshub_token}"
}

try:
    # Faz a requisição GET para a URL do dataset, enviando o token no cabeçalho
    response = requests.get(dataset_url, headers=headers)

    # Verifica se a requisição foi bem-sucedida (código 200 OK)
    response.raise_for_status()  # Isso vai gerar um erro se o download falhar (ex: token inválido, URL errada)

    # O 'response.content' contém os dados brutos do arquivo CSV.
    # Usamos 'io.StringIO' para que o pandas possa ler esses dados como se fossem um arquivo.
    # O 'decode('utf-8')' converte os bytes para uma string.
    csv_content = response.content.decode('utf-8')

    # Carrega o conteúdo do CSV em um DataFrame do pandas
    df = pd.read_csv(io.StringIO(csv_content))

    # Exibe as primeiras linhas para verificação
    print("DataFrame carregado com sucesso!")
    print(df.head())

except requests.exceptions.RequestException as e:
    print(f"Ocorreu um erro ao tentar baixar o arquivo: {e}")
    print("Por favor, verifique se a URL de download e o token de acesso estão corretos.")

except Exception as e:
    print(f"Ocorreu um erro inesperado: {e}")



Output()

Dataset URL: https://dagshub.com/api/v1/repos/estrellacouto05/quantum-finance-credit-score/raw/main/data/processed/credit_score_processed.csv
DataFrame carregado com sucesso!
   idade  renda_anual  salario_liquido_mensal  qtd_contas_bancarias  \
0   23.0     19114.12             1824.843333                   3.0   
1   23.0     19114.12             3335.886667                   3.0   
2   35.0     19114.12             3335.886667                   3.0   
3   23.0     19114.12             3335.886667                   3.0   
4   23.0     19114.12             1824.843333                   3.0   

   qtd_cartoes_credito  taxa_juros  qtd_emprestimos  dias_atraso_pagamento  \
0                    4         3.0              4.0                      3   
1                    4         3.0              4.0                      1   
2                    4         3.0              4.0                      3   
3                    4         3.0              4.0                      5   
4       

# Desenvolvimento e experimentos de modelos

## Inicialização do Dagshub com MLflow

In [7]:
# Inicializa o Dagshub com integração ao MLflow
dagshub.init(repo_owner='estrellacouto05',
             repo_name='quantum-finance-credit-score',
             mlflow=True)

print("Dagshub inicializado e MLflow configurado.")


Initialized MLflow to track repo "estrellacouto05/quantum-finance-credit-score"

Repository estrellacouto05/quantum-finance-credit-score initialized!

Dagshub inicializado e MLflow configurado.


In [8]:
# Ativa o registro automático de experimentos com MLflow
mlflow.autolog()

print("MLflow Autolog habilitado.")


2025/09/14 21:53:52 INFO mlflow.tracking.fluent: Autologging successfully enabled for lightgbm.
2025/09/14 21:53:54 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/09/14 21:53:54 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.


MLflow Autolog habilitado.


## Separação de Features e Target



Nesta etapa, vamos separar as variáveis independentes (features) do target (`score_credito`).  
Isso organiza o dataset para os modelos de Machine Learning, que aprenderão a prever o target com base nas features.

In [9]:
# Lista todas as colunas e remove o target 'score_credito' da lista de features
features = list(df.columns)
features.remove("score_credito")

# Exibe o total de features selecionadas
print(f"Total de features: {len(features)}")


Total de features: 48


In [10]:
# Cria os DataFrames para features (X) e target (y)
X = df[features]
y = df["score_credito"]

# Exibe informações básicas para conferência
print("Dimensões de X:", X.shape)
print("Dimensões de y:", y.shape)


Dimensões de X: (100000, 48)
Dimensões de y: (100000,)


## Divisão dos Dados e Escalonamento

Nesta etapa, dividimos os dados em conjuntos de treino e teste para avaliar o desempenho dos modelos de forma justa.  
Utilizamos **70% para treino** e **30% para teste**.

Depois aplicamos o **RobustScaler**, que é ideal para dados com outliers, pois utiliza medianas e intervalos interquartis, evitando distorções.


In [11]:
# Divide os dados em treino (70%) e teste (30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("Conjuntos criados:")
print(f"X_train: {X_train.shape}, X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}, y_test: {y_test.shape}")


Conjuntos criados:
X_train: (70000, 48), X_test: (30000, 48)
y_train: (70000,), y_test: (30000,)


In [12]:
# Inicializa o RobustScaler
scaler = RobustScaler()

# Ajusta o scaler apenas com os dados de treino e transforma
X_train_scaled = scaler.fit_transform(X_train)

# Aplica a mesma transformação nos dados de teste
X_test_scaled = scaler.transform(X_test)

print("Escalonamento concluído.")
print("Dimensões após escalonamento:", X_train_scaled.shape, X_test_scaled.shape)


Escalonamento concluído.
Dimensões após escalonamento: (70000, 48) (30000, 48)


In [13]:
X_train = X_train_scaled
X_test = X_test_scaled

## Função de Avaliação e Log no MLflow (Classificação)

- **Métricas principais:** precision, recall e f1-score para cada classe.
- **Destaque do recall da classe 0 (Poor)**, que é a mais importante.
- **Matriz de confusão** gerada como gráfico e registrada no MLflow como artefato.
- **Registro do modelo** no MLflow (CatBoost, XGBoost, LightGBM ou sklearn).

In [14]:
def evaluate_and_log_model(kind, model_name, model, X_test, y_test):
    # Faz previsões
    predictions = model.predict(X_test)

    # Calcula métricas de classificação
    training_accuracy_score_manual = model.score(X_train, y_train)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, predictions, labels=[0,1,2], zero_division=0)
    report = classification_report(y_test, predictions, digits=3)

    # Log de métricas macro (média das classes)
    mlflow.log_metric("Precision_macro", precision.mean())
    mlflow.log_metric("Recall_macro", recall.mean())
    mlflow.log_metric("F1_macro", f1.mean())

    # Log da acurácia de treino
    mlflow.log_metric("training_accuracy_score_manual", training_accuracy_score_manual)


    # Log detalhado de métricas por classe
    mlflow.log_metric("Recall_class_0_Poor", recall[0])
    mlflow.log_metric("Precision_class_0_Poor", precision[0])
    mlflow.log_metric("F1_class_0_Poor", f1[0])
    mlflow.log_metric("Recall_class_1_Standard", recall[1])
    mlflow.log_metric("Recall_class_2_Good", recall[2])

    # Cria assinatura do modelo para salvar no MLflow
    signature = infer_signature(X_test, predictions)

    # Salva o modelo de acordo com o tipo
    if kind == "catboost":
        mlflow.catboost.log_model(model, model_name, signature=signature, input_example=X_test[:5])
    elif kind == "xgboost":
        mlflow.xgboost.log_model(model, model_name, signature=signature, input_example=X_test[:5])
    elif kind == "lightgbm":
        mlflow.lightgbm.log_model(model, model_name, signature=signature, input_example=X_test[:5])
    else:
        mlflow.sklearn.log_model(model, model_name, signature=signature, input_example=X_test[:5])

    # Gera matriz de confusão
    cm = confusion_matrix(y_test, predictions, labels=[0,1,2])
    plt.figure(figsize=(6,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Poor', 'Standard', 'Good'], yticklabels=['Poor', 'Standard', 'Good'])
    plt.xlabel('Previsto')
    plt.ylabel('Real')
    plt.title(f'Matriz de Confusão - {model_name}')

    # Salva a figura temporariamente e envia como artefato para o MLflow
    with tempfile.TemporaryDirectory() as tmpdir:
        cm_path = os.path.join(tmpdir, "confusion_matrix.png")
        plt.savefig(cm_path)
        plt.close()
        mlflow.log_artifact(cm_path, artifact_path="confusion_matrix")

    # Print no console
    print(f"=== Avaliação do Modelo: {model_name} ===")
    print(report)
    print(f"Recall da classe 0 (Poor): {recall[0]:.3f}")
    print(f"Acurácia de Treino: {training_accuracy_score_manual:.3f}")


# Primeira Etapa de Treinamento de Modelos

A partir deste ponto, iniciamos o ciclo de **treinamento de modelos com MLflow**.

## XGBoost
  
Começamos com o **XGBoost Classifier**, um modelo de boosting altamente eficaz em dados tabulares.

- Foi usado **RandomizedSearchCV** com 30 combinações aleatórias de hiperparâmetros.
- Todas as métricas (precision, recall, f1-score) e a matriz de confusão foram registradas no MLflow.
- Nosso foco principal é **maximizar o recall da classe 0 (Poor)**, já que é a categoria mais crítica para o projeto.

Após avaliar o XGBoost, avançaremos para **LightGBM** e **CatBoost** para comparar os resultados.


In [ ]:
with mlflow.start_run(run_name="XGBoost_Classifier_RandomSearch"):

    # Definindo a grade de hiperparâmetros ampliada
    param_distributions = {
        'n_estimators': [50, 100, 200, 300],
        'max_depth': [3, 5, 7, 9],
        'learning_rate': [0.01, 0.05, 0.1],
        'subsample': [0.7, 0.8, 1.0],
        'colsample_bytree': [0.7, 0.8, 1.0],
        'min_child_weight': [1, 3, 5]
    }

    # Modelo base XGBoost
    xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)

    # RandomizedSearchCV para explorar combinações de hiperparâmetros
    random_search = RandomizedSearchCV(
        estimator=xgb,
        param_distributions=param_distributions,
        n_iter=30,                # número de combinações aleatórias a testar
        scoring='recall_macro',   # métrica principal
        cv=5,
        n_jobs=-1,
        verbose=1,
        random_state=42
    )

    # Treina com os dados escalonados
    random_search.fit(X_train_scaled, y_train)
    best_model = random_search.best_estimator_

    # Log dos melhores parâmetros no MLflow
    best_params = random_search.best_params_
    for param, value in best_params.items():
        mlflow.log_param(param, value)

    # Avalia e registra o modelo
    evaluate_and_log_model("xgboost", "XGBoost Classifier RandomSearch", best_model, X_test_scaled, y_test)

2025/08/03 22:02:56 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Fitting 5 folds for each of 30 candidates, totalling 150 fits


2025/08/03 22:13:25 INFO mlflow.sklearn.utils: Logging the 5 best runs, 25 runs will be omitted.


=== Avaliação do Modelo: XGBoost Classifier RandomSearch ===
              precision    recall  f1-score   support

           0      0.767     0.727     0.747      8805
           1      0.763     0.795     0.779     15873
           2      0.669     0.642     0.655      5322

    accuracy                          0.748     30000
   macro avg      0.733     0.721     0.727     30000
weighted avg      0.747     0.748     0.747     30000

Recall da classe 0 (Poor): 0.727
🏃 View run XGBoost_Classifier_RandomSearch at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/f9ec195a72dd4afb9b91f99ef5f727b2
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


## LightGBM

Agora vamos treinar o **LightGBM Classifier**, outro modelo de boosting eficiente e mais rápido que o XGBoost em muitos cenários.

- Usaremos **RandomizedSearchCV** com 30 combinações, igual ao XGBoost, para manter o tempo de treino semelhante (~150 fits).
- Vamos incluir parâmetros específicos do LightGBM, como **num_leaves**, que controla a complexidade das árvores.
- Métrica de busca: **recall_macro**, priorizando melhor cobertura da classe “Poor”.
- Tudo será registrado no **MLflow** para comparação posterior.

In [ ]:
with mlflow.start_run(run_name="LightGBM_Classifier_RandomSearch"):

    # Definição do espaço de busca
    param_distributions = {
        'n_estimators': [50, 100, 200, 300],
        'max_depth': [3, 5, 7, 9, -1],  # -1 = sem limite de profundidade
        'learning_rate': [0.01, 0.05, 0.1],
        'num_leaves': [15, 31, 63, 127],
        'subsample': [0.7, 0.8, 1.0],
        'colsample_bytree': [0.7, 0.8, 1.0]
    }

    # Modelo base LightGBM
    lgb = LGBMClassifier(objective='multiclass', num_class=3, random_state=42)

    # RandomizedSearchCV – 30 combinações x 5 folds = 150 fits
    random_search = RandomizedSearchCV(
        estimator=lgb,
        param_distributions=param_distributions,
        n_iter=30,
        scoring='recall_macro',
        cv=5,
        n_jobs=-1,
        verbose=1,
        random_state=42
    )

    # Treina o modelo
    random_search.fit(X_train_scaled, y_train)
    best_model = random_search.best_estimator_

    # Log dos melhores hiperparâmetros
    best_params = random_search.best_params_
    for param, value in best_params.items():
        mlflow.log_param(param, value)

    # Avalia e registra o modelo no MLflow
    evaluate_and_log_model("lightgbm", "LightGBM Classifier RandomSearch", best_model, X_test_scaled, y_test)


2025/08/03 22:21:43 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Fitting 5 folds for each of 30 candidates, totalling 150 fits
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010766 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2708
[LightGBM] [Info] Number of data points in the train set: 70000, number of used features: 48
[LightGBM] [Info] Start training from score -1.243159
[LightGBM] [Info] Start training from score -0.629475
[LightGBM] [Info] Start training from score -1.722287


2025/08/03 22:30:14 INFO mlflow.sklearn.utils: Logging the 5 best runs, 25 runs will be omitted.


=== Avaliação do Modelo: LightGBM Classifier RandomSearch ===
              precision    recall  f1-score   support

           0      0.781     0.778     0.779      8805
           1      0.788     0.810     0.799     15873
           2      0.748     0.692     0.719      5322

    accuracy                          0.780     30000
   macro avg      0.773     0.760     0.766     30000
weighted avg      0.779     0.780     0.779     30000

Recall da classe 0 (Poor): 0.778
🏃 View run LightGBM_Classifier_RandomSearch at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/eaf41b22355a40d9a6a6b939818b364d
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


Melhor foi light gbm

##  Random Forest Classifier

Neste experimento, aplicamos o modelo `RandomForestClassifier` para classificação dos scores de crédito, utilizando `RandomizedSearchCV` com validação cruzada (CV=5) para otimização de hiperparâmetros.

A escolha pelo `Random Forest` visa avaliar o desempenho de um modelo de ensemble tradicional, mais interpretável, como alternativa aos métodos baseados em Gradient Boosting (XGBoost e LightGBM).

**Configurações do experimento:**
- 50 combinações testadas com `RandomizedSearchCV` (`n_iter=50`)
- Métrica de avaliação principal: `recall_macro`
- Hiperparâmetros otimizados: número de estimadores, profundidade máxima (com limite para evitar overfitting), amostragem mínima para splits e folhas, tipo de critério, entre outros.
- Avaliação com função customizada `evaluate_and_log_model`, registrando métricas detalhadas (precision, recall, f1-score, matriz de confusão), com foco no `recall` da classe 0 (`Poor`), além de logging via MLflow.

O objetivo é verificar se o Random Forest entrega resultados competitivos e interpretar seu comportamento em comparação com os modelos anteriores.


In [ ]:
# Início do experimento com MLflow
with mlflow.start_run(run_name="RandomForest_Classifier_RandomSearch"):

    # Instancia o modelo base
    rf = RandomForestClassifier(random_state=42)

    # Espaço de busca ajustado (sem max_depth=None para evitar overfitting)
    param_distributions = {
        'n_estimators': [50, 100, 200],
        'max_depth': [5, 10, 15, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['auto', 'sqrt', 'log2'],
        'bootstrap': [True, False],
        'criterion': ['gini', 'entropy']
    }

    # RandomizedSearchCV com 50 combinações
    random_search = RandomizedSearchCV(
        estimator=rf,
        param_distributions=param_distributions,
        n_iter=50,
        scoring='recall_macro',
        cv=5,
        n_jobs=-1,
        verbose=1,
        random_state=42
    )

    # Treinamento
    random_search.fit(X_train_scaled, y_train)

    # Modelo final
    best_model = random_search.best_estimator_

    # Log dos melhores hiperparâmetros
    best_params = random_search.best_params_
    for param, value in best_params.items():
        mlflow.log_param(param, value)

    # Avaliação + log via função customizada
    evaluate_and_log_model("sklearn", "RandomForest Classifier RandomSearch", best_model, X_test_scaled, y_test)

2025/08/05 11:25:15 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Fitting 5 folds for each of 50 candidates, totalling 250 fits


2025/08/05 12:00:24 INFO mlflow.sklearn.utils: Logging the 5 best runs, 45 runs will be omitted.


🏃 View run auspicious-deer-16 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/702b80c4c87b4456a148eb28e2e1b791
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run monumental-ox-450 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/83a8a9a3b726467fa8b2980745aa22ca
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run grandiose-zebra-368 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/8f247204dc2d4f8a9a0ff3d2547aa025
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run burly-ape-995 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/e8bf5ee20de9424099f1d17821de5018
🧪 View experiment at: https://dagsh

=== Avaliação do Modelo: RandomForest Classifier RandomSearch ===
              precision    recall  f1-score   support

           0      0.780     0.754     0.767      8805
           1      0.778     0.807     0.793     15873
           2      0.713     0.673     0.693      5322

    accuracy                          0.768     30000
   macro avg      0.757     0.745     0.751     30000
weighted avg      0.767     0.768     0.767     30000

Recall da classe 0 (Poor): 0.754
🏃 View run RandomForest_Classifier_RandomSearch at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/df711845800e452aa800d9578ce58457
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


🏃 View run enchanting-shad-321 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/1ab7344499f74817a85e876013514d9f
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run serious-asp-250 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/c796d96f67054d22871e19825ce2175d
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run invincible-rook-21 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/22ef267fe2ca4032a2ddcce5d44abc9a
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run angry-asp-955 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/7d9fc16a65f342b798a7e2299bdbfabd
🧪 View experiment at: https://dagshub

🏃 View run worried-steed-201 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/09b6d6b37e724f3a8a1da73fa71fc074
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run chill-mare-104 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/1ff41165c2554efe88c2927706ff5571
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run chill-smelt-845 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/e221b0dce6884322ad9a016113666026
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run big-asp-866 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/7bead2d39d444bf188e4735a0c8630a6
🧪 View experiment at: https://dagshub.com/est

## Ultima

In [17]:
# Início do experimento com MLflow
with mlflow.start_run(run_name="LGBM_Classifier_RandomSearch_LowCost"):

    # Instancia o modelo base com configurações fixas
    lgbm = LGBMClassifier(
        objective='multiclass',
        num_class=3,          # Ajuste se o número de classes for diferente
        random_state=42,
        n_jobs=-1
    )

    # Espaço de busca mais enxuto e direcionado para 30 iterações
    param_distributions = {
        # 1. Controle de Complexidade (Parâmetros de maior impacto)
        'num_leaves': [25, 31, 45, 55],
        'max_depth': [6, 8, 10, 12],

        # 2. Regularização (Valores discretos e impactantes)
        'reg_alpha': [0.1, 0.5, 1.0, 1.5],
        'reg_lambda': [0.1, 0.5, 1.0, 1.5],
        'min_child_samples': [20, 40, 60],

        # 3. Aleatoriedade (Valores comuns e eficazes)
        'subsample': [0.7, 0.8, 0.9],
        'colsample_bytree': [0.7, 0.8, 0.9],

        # 4. Processo de Treinamento (Valores fixos para simplificar a busca)
        'learning_rate': [0.05], # Fixo em um valor bom e seguro
        'n_estimators': [400]    # Fixo em um valor razoável (use com Early Stopping se possível)
    }

    # RandomizedSearchCV com no máximo 30 combinações
    random_search = RandomizedSearchCV(
        estimator=lgbm,
        param_distributions=param_distributions,
        n_iter=30,  # <<< Limite de 30 treinos, como solicitado
        scoring='recall_macro',
        cv=5,
        n_jobs=-1,
        verbose=1,
        random_state=42
    )

    # Treinamento
    # Para otimizar ainda mais, considere adicionar um callback de Early Stopping aqui, se seu
    # ambiente permitir (ex: passando eval_set e callbacks para o .fit()).
    random_search.fit(X_train_scaled, y_train)

    # Modelo final com os melhores parâmetros
    best_model = random_search.best_estimator_

    # Log dos melhores hiperparâmetros no MLflow
    best_params = random_search.best_params_
    mlflow.log_params(best_params)
    mlflow.log_metric("best_cv_score", random_search.best_score_)

    print("\nMelhores Hiperparâmetros Encontrados:")
    print(best_params)
    print(f"\nMelhor score de CV (recall_macro): {random_search.best_score_:.4f}")

    # Avaliação do modelo final no conjunto de teste e log no MLflow
    evaluate_and_log_model("lightgbm", "LGBM Classifier RandomSearch LowCost", best_model, X_test_scaled, y_test)


2025/09/15 12:19:59 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Fitting 5 folds for each of 30 candidates, totalling 150 fits
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.028578 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2708
[LightGBM] [Info] Number of data points in the train set: 70000, number of used features: 48
[LightGBM] [Info] Start training from score -1.243159
[LightGBM] [Info] Start training from score -0.629475
[LightGBM] [Info] Start training from score -1.722287
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

2025/09/15 12:39:26 INFO mlflow.sklearn.utils: Logging the 5 best runs, 25 runs will be omitted.


🏃 View run gifted-elk-275 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/d4bf9908c90543ee90ff5f9ab41beeaf
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


🏃 View run gregarious-koi-312 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/943be36eb3e24db296d97cabe8962645
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0

Melhores Hiperparâmetros Encontrados:
{'subsample': 0.9, 'reg_lambda': 0.5, 'reg_alpha': 0.1, 'num_leaves': 55, 'n_estimators': 400, 'min_child_samples': 60, 'max_depth': 10, 'learning_rate': 0.05, 'colsample_bytree': 0.7}

Melhor score de CV (recall_macro): 0.7227


=== Avaliação do Modelo: LGBM Classifier RandomSearch LowCost ===
              precision    recall  f1-score   support

           0      0.764     0.721     0.742      8805
           1      0.760     0.799     0.779     15873
           2      0.692     0.652     0.672      5322

    accuracy                          0.750     30000
   macro avg      0.739     0.724     0.731     30000
weighted avg      0.749     0.750     0.749     30000

Recall da classe 0 (Poor): 0.721
Acurácia de Treino: 0.829
🏃 View run LGBM_Classifier_RandomSearch_LowCost at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/d4082405211c46c5af186ef93c1a4bcf
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


## Fine-Tuning do LightGBM com RandomizedSearchCV

Nesta etapa, realizamos um ajuste fino (fine-tuning) no modelo LightGBM com foco em melhorar o **recall da classe 0 (Poor)**.  
Baseado nos melhores parâmetros anteriores, restringimos os ranges de busca para explorar variações próximas e mais promissoras.  
Usamos `RandomizedSearchCV` com `n_iter=50` e validação cruzada (`cv=5`), mantendo o controle de overfitting por meio de `num_leaves`, `subsample` e `colsample_bytree`.  
Os resultados serão registrados automaticamente no MLflow via integração com o Dagshub.


In [ ]:
with mlflow.start_run(run_name="LightGBM_FineTuning"):

    model = LGBMClassifier(objective='multiclass', num_class=3, random_state=42)

    param_distributions = {
        'n_estimators': [300, 400, 500],
        'learning_rate': [0.05, 0.08, 0.1],
        'num_leaves': [95, 127, 150],
        'max_depth': [5, 7, 9, -1],
        'subsample': [0.6, 0.7, 0.8],
        'colsample_bytree': [0.7, 0.85, 1.0]
    }

    randomized_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_distributions,
        n_iter=50,
        scoring='recall_macro',
        cv=5,
        random_state=42,
        verbose=1,
        n_jobs=-1
    )

    randomized_search.fit(X_train_scaled, y_train)
    best_model = randomized_search.best_estimator_

    # Loga os melhores hiperparâmetros
    mlflow.log_params(randomized_search.best_params_)

    # Avalia e registra o modelo no MLflow
    evaluate_and_log_model("lightgbm", "LightGBM FineTuned", best_model, X_test_scaled, y_test)

2025/08/05 12:35:54 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Fitting 5 folds for each of 50 candidates, totalling 250 fits
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008393 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2708
[LightGBM] [Info] Number of data points in the train set: 70000, number of used features: 48
[LightGBM] [Info] Start training from score -1.243159
[LightGBM] [Info] Start training from score -0.629475
[LightGBM] [Info] Start training from score -1.722287


2025/08/05 13:25:15 INFO mlflow.sklearn.utils: Logging the 5 best runs, 45 runs will be omitted.


=== Avaliação do Modelo: LightGBM FineTuned ===
              precision    recall  f1-score   support

           0      0.785     0.786     0.785      8805
           1      0.793     0.812     0.803     15873
           2      0.755     0.701     0.727      5322

    accuracy                          0.784     30000
   macro avg      0.778     0.766     0.772     30000
weighted avg      0.784     0.784     0.784     30000

Recall da classe 0 (Poor): 0.786
🏃 View run LightGBM_FineTuning at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/05cfe5fa3fa544bbba01799cb7f7b903
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


### Fine-tuning adicional no LightGBM com foco em `n_estimators` e `learning_rate`

Neste experimento, mantemos os melhores hiperparâmetros encontrados anteriormente (como `num_leaves`, `max_depth`, `subsample` e `colsample_bytree`) e realizamos um ajuste fino especificamente sobre os parâmetros `n_estimators` e `learning_rate`.

O objetivo é verificar se o aumento do número de estimadores (de 500 para 750 e 1000) combinado com taxas de aprendizado mais baixas (0.05, 0.03 e 0.01) resulta em ganho de recall, principalmente da classe 0 (Poor), sem causar overfitting.

A técnica continua sendo o `RandomizedSearchCV` com 50 iterações e validação cruzada (cv=5), mantendo o padrão adotado em experimentos anteriores.



In [ ]:
with mlflow.start_run(run_name="LightGBM_Estimators_LearningRate_Tuning"):

    model = LGBMClassifier(objective='multiclass', num_class=3, random_state=42)

    param_distributions = {
        'n_estimators': [500, 750, 1000],
        'learning_rate': [0.05, 0.03, 0.01],
        'num_leaves': [127],
        'max_depth': [-1],
        'subsample': [0.7],
        'colsample_bytree': [1.0]
    }

    randomized_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_distributions,
        n_iter=50,
        scoring='recall_macro',
        cv=5,
        random_state=42,
        verbose=1,
        n_jobs=-1
    )

    randomized_search.fit(X_train_scaled, y_train)
    best_model = randomized_search.best_estimator_

    # Log dos principais hiperparâmetros selecionados
    mlflow.log_param("best_n_estimators", best_model.n_estimators)
    mlflow.log_param("best_learning_rate", best_model.learning_rate)

    # Avaliação e log do modelo
    evaluate_and_log_model("lightgbm", "LightGBM Estimators-LR Tuning", best_model, X_test_scaled, y_test)

2025/08/05 13:57:07 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Fitting 5 folds for each of 9 candidates, totalling 45 fits
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.024234 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2708
[LightGBM] [Info] Number of data points in the train set: 70000, number of used features: 48
[LightGBM] [Info] Start training from score -1.243159
[LightGBM] [Info] Start training from score -0.629475
[LightGBM] [Info] Start training from score -1.722287


2025/08/05 14:20:52 INFO mlflow.sklearn.utils: Logging the 5 best runs, 4 runs will be omitted.


=== Avaliação do Modelo: LightGBM Estimators-LR Tuning ===
              precision    recall  f1-score   support

           0      0.785     0.789     0.787      8805
           1      0.794     0.812     0.803     15873
           2      0.759     0.701     0.729      5322

    accuracy                          0.786     30000
   macro avg      0.779     0.767     0.773     30000
weighted avg      0.785     0.786     0.785     30000

Recall da classe 0 (Poor): 0.789
🏃 View run LightGBM_Estimators_LearningRate_Tuning at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/eb3c5dfe89f44412971d50969d0c882f
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


###  Escolha final do modelo LightGBM (Fine-Tuning)

Após dois ciclos de fine-tuning com o LightGBM, foram comparadas duas configurações principais:

| Modelo | `n_estimators` | `learning_rate` | Recall Classe 0 (Poor) |
|--------|----------------|------------------|--------------------------|
| A (1º Fine-tune) | **500** | **0.1** | **0.786** |
| B (2º Fine-tune) | **1000** | **0.05** | **0.789** |

Apesar do modelo B utilizar o dobro de árvores com um `learning_rate` mais baixo (0.05), **a melhora no recall da classe 0 foi mínima** (de 0.786 para 0.789), sem ganho significativo em outras métricas. Além disso, esse aumento nos estimadores **eleva o custo computacional e o risco de overfitting** sem retorno proporcional em desempenho.

####  Conclusão:
Optamos por manter o modelo com:
- `n_estimators = 500`
- `learning_rate = 0.1`

Essa combinação mostrou-se mais eficiente, com ótimo desempenho geral, menor complexidade e melhor equilíbrio entre performance e custo de treinamento.




## CatBoost

Agora vamos treinar o **CatBoost Classifier**, um modelo de boosting desenvolvido pela Yandex com excelente desempenho em bases com variáveis categóricas e estrutura mista (numéricas + booleanas).

- Usaremos o **RandomizedSearchCV** com 30 combinações, mantendo o padrão de experimentação adotado nos modelos anteriores.
- Serão explorados hiperparâmetros como **depth**, **learning_rate**, **l2_leaf_reg** e **border_count**.
- A métrica de otimização interna será **TotalF1**, adequada para tarefas multiclasse, enquanto a métrica de avaliação externa continua sendo o **recall_macro**.
- Toda a execução será rastreada e registrada no **MLflow**, com foco na classe “Poor” e comparação direta com os demais modelos.


In [ ]:
catboost_temp_dir = mkdtemp()

with mlflow.start_run(run_name="CatBoost_FineTuning"):

    model = CatBoostClassifier(
        loss_function='MultiClass',
        eval_metric='TotalF1',  #  métrica escalar compatível
        verbose=0,
        train_dir=catboost_temp_dir,
        random_state=42
    )

    param_distributions = {
        'iterations': [300, 500, 750],
        'learning_rate': [0.01, 0.05, 0.1],
        'depth': [4, 6, 8, 10],
        'l2_leaf_reg': [1, 3, 5, 7],
        'border_count': [32, 64, 128]
    }

    randomized_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_distributions,
        n_iter=30,
        scoring='recall_macro',  # usado para selecionar o melhor modelo
        cv=5,
        verbose=1,
        n_jobs=1,
        random_state=42
    )

    randomized_search.fit(X_train, y_train)

    best_model = randomized_search.best_estimator_

    # Log dos hiperparâmetros
    mlflow.log_param("best_iterations", best_model.get_params()["iterations"])
    mlflow.log_param("best_learning_rate", best_model.get_params()["learning_rate"])
    mlflow.log_param("best_depth", best_model.get_params()["depth"])
    mlflow.log_param("best_l2_leaf_reg", best_model.get_params()["l2_leaf_reg"])
    mlflow.log_param("best_border_count", best_model.get_params()["border_count"])

    # Avaliação
    evaluate_and_log_model("catboost", "CatBoost FineTuned", best_model, X_test, y_test)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


2025/08/05 18:17:53 INFO mlflow.sklearn.utils: Logging the 5 best runs, 25 runs will be omitted.


🏃 View run trusting-stoat-841 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/65d6e49d1b96461b845982489c694680
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run victorious-skunk-835 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/59239c09c853459c9889f22835e8195e
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run receptive-yak-708 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/ed4a8cce2fdf46fcb168768bb4ff358e
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run adaptable-finch-354 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/3738e2e53f624e8d9df1a46d6e113ddb
🧪 View experiment at: https:

=== Avaliação do Modelo: CatBoost FineTuned ===
              precision    recall  f1-score   support

           0      0.771     0.743     0.756      8805
           1      0.766     0.808     0.786     15873
           2      0.719     0.644     0.680      5322

    accuracy                          0.760     30000
   macro avg      0.752     0.732     0.741     30000
weighted avg      0.759     0.760     0.759     30000

Recall da classe 0 (Poor): 0.743
🏃 View run CatBoost_FineTuning at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/56d42725396f4f47b84222b54a4b5b1e
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


## Stacking Ensemble – LightGBM + CatBoost

Após avaliarmos isoladamente os modelos **LightGBM** e **CatBoost**, partiremos agora para a construção de um **modelo em ensemble via StackingClassifier**, com o objetivo de combinar suas forças.

- O **Stacking** é uma técnica de ensemble que combina diferentes algoritmos como base learners e utiliza um modelo meta para agregar suas previsões.
- Utilizaremos o **LightGBM** e o **CatBoost**, ambos já ajustados com **RandomizedSearchCV**, como base learners do ensemble.
- O meta-modelo escolhido será uma **árvore de decisão simples**, que aprenderá a combinar as predições das bases.
- Optamos por `passthrough=False` para usar apenas as predições dos modelos base como entrada para o meta-modelo, reduzindo risco de overfitting.
- O treinamento será feito sobre a mesma divisão `train/test`, e também avaliaremos a **robustez via validação cruzada externa** (5-fold).
- Toda a execução será rastreada pelo **MLflow**, incluindo métricas, matriz de confusão e parâmetros.

Essa abordagem visa obter um modelo mais generalizável e robusto, maximizando o desempenho sobre as diferentes classes.



In [ ]:
with mlflow.start_run(run_name="Stacking_LGBM_CatBoost_RFMeta"):

    # Modelos base com hiperparâmetros ajustados
    estimators = [
        ("lgbm", LGBMClassifier(
            n_estimators=500,
            learning_rate=0.1,
            num_leaves=127,
            max_depth=-1,
            subsample=0.7,
            colsample_bytree=1.0,
            objective='multiclass',
            num_class=3,
            random_state=42
        )),
        ("catboost", CatBoostClassifier(
            iterations=750,
            learning_rate=0.05,
            depth=10,
            l2_leaf_reg=3,
            border_count=64,
            loss_function='MultiClass',
            eval_metric='TotalF1',
            verbose=0,
            random_state=42
        ))
    ]

    # Meta-modelo (estimador final)
    final_estimator = RandomForestClassifier(
        n_estimators=100,
        max_depth=3,
        random_state=42
    )

    # Definição do ensemble com passthrough=False
    stacking_clf = StackingClassifier(
        estimators=estimators,
        final_estimator=final_estimator,
        cv=5,
        n_jobs=-1,
        passthrough=False
    )

    # Treinamento
    stacking_clf.fit(X_train, y_train)

    # Avaliação com função central
    evaluate_and_log_model("stacking", "Stacking LGBM + CatBoost", stacking_clf, X_test, y_test)

    # Log extra
    mlflow.log_param("ensemble_method", "stacking")
    mlflow.log_param("meta_estimator", "random_forest")
    mlflow.log_param("passthrough", False)
    mlflow.log_param("cv", 5)


=== Avaliação do Modelo: Stacking LGBM + CatBoost ===
              precision    recall  f1-score   support

           0      0.768     0.842     0.804      8805
           1      0.833     0.773     0.802     15873
           2      0.727     0.768     0.747      5322

    accuracy                          0.792     30000
   macro avg      0.776     0.794     0.784     30000
weighted avg      0.795     0.792     0.793     30000

Recall da classe 0 (Poor): 0.842
🏃 View run Stacking_LGBM_CatBoost_RFMeta at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/c85566115777495aa40d6814d0bb7102
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


### Justificativa para o Uso do Ensemble LightGBM + CatBoost

A decisão por utilizar um Stacking com **LightGBM** e **CatBoost** foi baseada nos resultados individuais obtidos após tuning cuidadoso:

- O **CatBoost FineTuned** obteve excelente performance, com **recall de 0.743 na classe 0 (Poor)** e uma média geral (`recall_macro`) de **0.732**, superando todos os modelos anteriores.
- O **LightGBM FineTuned** também apresentou bom desempenho, com **boa cobertura da classe 2 (Good)** e um tempo de treinamento significativamente inferior.
- Já o Stacking inicial com **três modelos (LightGBM, XGBoost e RandomForest)** teve desempenho muito abaixo do esperado, com recall da classe 0 em apenas 0.33 — sendo **descartado** por baixa eficácia e sobreposição de comportamento entre os modelos.
- Ao combinar apenas **os dois melhores modelos (CatBoost e LightGBM)**, alcançamos um **recall da classe 0 de 0.842** e um **recall_macro de 0.794**, além de uma acurácia de **0.792**.

Esse resultado evidencia que o ensemble entre esses dois algoritmos **potencializou os pontos fortes de cada um** e contribuiu para **uma classificação mais equilibrada entre as três classes**.

Com isso, este modelo passa a ser o **candidato principal à produção**, e será submetido à validação cruzada externa antes de ser registrado no **MLflow Model Registry**.


In [ ]:
from sklearn.model_selection import cross_validate

# Métricas para avaliação externa
scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']

# Aplicando validação cruzada externa ao modelo já ajustado
cv_results = cross_validate(
    estimator=stacking_clf,
    X=X_train,
    y=y_train,
    cv=5,
    scoring=scoring,
    return_train_score=False,
    n_jobs=-1
)

# Exibindo resultados médios e variabilidade
print("🔎 Resultados da Validação Cruzada Externa (Stacking Final):\n")
for metric in scoring:
    media = cv_results[f'test_{metric}'].mean()
    desvio = cv_results[f'test_{metric}'].std()
    print(f"{metric:<18}: {media:.4f} ± {desvio:.4f}")


🔎 Resultados da Validação Cruzada Externa (Stacking Final):

accuracy          : 0.7810 ± 0.0033
precision_macro   : 0.7641 ± 0.0040
recall_macro      : 0.7801 ± 0.0047
f1_macro          : 0.7712 ± 0.0043


### Validação Cruzada Externa – Stacking LightGBM + CatBoost

Para garantir a robustez do ensemble construído com **LightGBM + CatBoost**, foi aplicada uma **validação cruzada externa (5-fold)** utilizando o `cross_validate` do `sklearn`.

- Essa abordagem avalia o modelo com diferentes divisões do dataset, oferecendo uma visão mais confiável sobre sua **generalização**.
- Foram calculadas as métricas: **accuracy**, **precision_macro**, **recall_macro** e **f1_macro**.
- Os resultados obtidos confirmam a **estabilidade e eficácia do ensemble**, com variações pequenas entre os folds.

###  Resultados da Validação Cruzada Externa (Stacking Final):

- **Accuracy**: `0.7810 ± 0.0033`  
- **Precision (Macro Avg)**: `0.7641 ± 0.0040`  
- **Recall (Macro Avg)**: `0.7801 ± 0.0047`  
- **F1-score (Macro Avg)**: `0.7712 ± 0.0043`

Esses resultados reforçam a escolha do modelo como **candidato final para produção**, com excelente equilíbrio entre as classes, especialmente em cenários multiclasse com forte desbalanceamento.


# Nova Experimentação treinamento para melhora do modelo



## Análise Resumida dos Modelos no DagsHub
Esta cédula de código busca e compara os melhores modelos do seu experimento MLflow no DagsHub.

O script carrega suas credenciais de forma segura do arquivo .env.

Em seguida, classifica os modelos com base no recall da classe Poor, a métrica mais importante para o projeto.

Por fim, ele extrai e exibe uma tabela com as métricas-chave (como acurácia de treino e teste) e os principais parâmetros para te ajudar a identificar o melhor modelo e a causa do overfitting.

In [ ]:

# Encontra e carrega o arquivo .env, buscando a partir do diretório atual
load_dotenv(find_dotenv())

# Agora você pode acessar as variáveis de ambiente
mlflow_user = os.environ.get('MLFLOW_TRACKING_USERNAME')
mlflow_token = os.environ.get('MLFLOW_TRACKING_PASSWORD')

# A URI de rastreamento segue o padrão: https://dagshub.com/<usuario>/<repo>.mlflow
# O autolog já faz essa configuração para você, mas é bom ter o comando explícito aqui
mlflow.set_tracking_uri("https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow")
# 2. Inicializa o cliente do MLflow
client = MlflowClient()

# Define o nome do experimento
# Verifique o nome exato no seu Dashboard do DagsHub
experiment_name = "Default" # Geralmente é 'Default', a menos que você tenha criado um nome específico

try:
    experiment = client.get_experiment_by_name(experiment_name)
    if experiment is None:
        raise ValueError(f"Experiment with name '{experiment_name}' not found.")

    # Busca as 7 melhores execuções baseadas no recall da classe 0 (Poor)
    runs = client.search_runs(
        experiment_ids=[experiment.experiment_id],
        order_by=["metrics.Recall_class_0_Poor DESC"],
        max_results=7
    )

    model_data = []

    for run in runs:
        model_name = run.data.tags.get('mlflow.runName', run.info.run_name)

        metrics = run.data.metrics
        recall_poor = metrics.get('Recall_class_0_Poor', 'N/A')

        # O MLflow Autolog pode ter registrado a acurácia de treino e teste
        training_accuracy = metrics.get('training_accuracy_score', 'N/A')
        test_accuracy = metrics.get('accuracy', 'N/A')

        params = run.data.params

        relevant_params = {
            'learning_rate': params.get('learning_rate', 'N/A'),
            'n_estimators': params.get('n_estimators', 'N/A'),
            'num_leaves': params.get('num_leaves', 'N/A'),
            'depth': params.get('depth', 'N/A'),
            'l2_leaf_reg': params.get('l2_leaf_reg', 'N/A'),
            'meta_estimator': params.get('meta_estimator', 'N/A')
        }

        model_data.append({
            'Modelo': model_name,
            'Recall (Poor)': recall_poor,
            'Acurácia de Treino': training_accuracy,
            'Acurácia de Teste': test_accuracy,
            'Parâmetros': relevant_params
        })

    df = pd.DataFrame(model_data)

    print("Top 7 Modelos por Recall da Classe 'Poor' no DagsHub")
    print("-" * 60)
    print(df.to_markdown(index=False))

except ValueError as e:
    print(f"Erro: {e}")
except Exception as e:
    print(f"Ocorreu um erro ao conectar ou buscar no MLflow: {e}")



Top 7 Modelos por Recall da Classe 'Poor' no DagsHub
------------------------------------------------------------
| Modelo                                  |   Recall (Poor) |   Acurácia de Treino | Acurácia de Teste   | Parâmetros                                                                                                                                    |
|:----------------------------------------|----------------:|---------------------:|:--------------------|:----------------------------------------------------------------------------------------------------------------------------------------------|
| Stacking_LGBM_CatBoost_RFMeta           |        0.842249 |             0.980586 | N/A                 | {'learning_rate': 'N/A', 'n_estimators': 'N/A', 'num_leaves': 'N/A', 'depth': 'N/A', 'l2_leaf_reg': 'N/A', 'meta_estimator': 'random_forest'} |
| LightGBM_Estimators_LearningRate_Tuning |        0.788984 |             0.996771 | N/A                 | {'learning_rate': 'N/A', '

## Stacking LGBM + Catboost: Melhora do modelo


### Stacking (LGBM + CatBoost) com meta-learner Logístico e busca aleatória

Estratégia: empilhar LGBM e CatBoost para capturar vieses complementares e reduzir erro sistemático por classe.
O meta-learner é uma Regressão Logística multinomial, escolhida por simplicidade, regularização e boa calibração de decisão.
Usamos RandomizedSearchCV para explorar hiperparâmetros-chave dos base learners e a regularização do meta, priorizando recall_macro.
A validação cruzada (cv=3) acelera a descoberta de direções promissoras; vencedores devem ser confirmados depois com cv=5.
O meta recebe apenas as previsões dos base models (passthrough=False), mitigando overfitting e focando a combinação das probabilidades.
O MLflow registra melhores parâmetros/score, permitindo comparar versões e orientar próximos passos (calibração e thresholds por classe).

In [ ]:
# Define os estimadores base
estimators = [
    ("lgbm", LGBMClassifier(random_state=42)),
    ("catboost", CatBoostClassifier(verbose=0, random_state=42))
]

# Define o meta-modelo
final_estimator = LogisticRegression(solver='lbfgs', multi_class='multinomial')

# Define o modelo de Stacking
stacking_clf = StackingClassifier(
    estimators=estimators,
    final_estimator=final_estimator,
    cv=5,
    n_jobs=-1
)

# Define a grade de parâmetros para a busca aleatória
param_dist = {
    'lgbm__n_estimators': np.arange(100, 1000, 100),
    'lgbm__learning_rate': [0.01, 0.05, 0.1, 0.2],
    'lgbm__num_leaves': [16, 31, 63, 127],
    'catboost__iterations': np.arange(100, 1000, 100),
    'catboost__learning_rate': [0.01, 0.05, 0.1, 0.2],
    'catboost__depth': [4, 6, 8, 10],
    'catboost__l2_leaf_reg': [1, 3, 5, 10],
    'final_estimator__C': [0.1, 1.0, 10.0]
}

# Configura o RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=stacking_clf,
    param_distributions=param_dist,
    n_iter=50,
    scoring='recall_macro',
    cv=3,
    n_jobs=-1,
    verbose=1,
    random_state=42
)

# Inicia o run no MLflow e executa a busca
with mlflow.start_run(run_name="Stacking_RandomSearch_Pre_Scaled_Data"):
    random_search.fit(X_train_scaled, y_train)

    # Log dos melhores parâmetros e métricas no MLflow
    mlflow.log_params(random_search.best_params_)
    mlflow.log_metric("best_recall_macro_score", random_search.best_score_)

    # Exibe os melhores parâmetros e a melhor pontuação
    print("Melhores parâmetros encontrados:")
    print(random_search.best_params_)
    print(f"Melhor pontuação (recall macro): {random_search.best_score_:.4f}")

    # Armazena o melhor modelo treinado
    best_stacking_model = random_search.best_estimator_

2025/08/27 16:09:01 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Fitting 3 folds for each of 50 candidates, totalling 150 fits


2025/08/28 02:07:28 INFO mlflow.sklearn.utils: Logging the 5 best runs, 45 runs will be omitted.


🏃 View run stately-wasp-233 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/f8142d7ae9b94f12aa778a340a89fed2
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run enchanting-worm-126 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/6941e0afad7b4e95ad3ca7e78b43a2ee
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run handsome-fish-556 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/5b0cb52c5b6c48f5b6a4fa1d4872a592
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run puzzled-wasp-608 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/d74a40c585fe440797909380818f5635
🧪 View experiment at: https://dags

🏃 View run bustling-hawk-256 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/cc878ab7d37c46fbaefa4689b06a1260
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


🏃 View run hilarious-bug-345 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/897ec702ca2d4e84bb48adc7c099f07a
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run wise-conch-344 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/9cbc6cbc4bc4426992d00bc41cc57196
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run bittersweet-cat-344 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/b34d53303ae0451281075ebee64f5d2d
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run youthful-deer-665 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/fcea06b95d1042758ed65b9b796d3a15
🧪 View experiment at: https://dagsh

🏃 View run big-dove-219 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/c8053fd442524b8c8e817dc96a51893c
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run judicious-sow-660 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/0c77e6092fab4da1927676de394e9bf0
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run welcoming-kite-36 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/ef39264abe31498e80332fcedb83d448
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run wise-wolf-760 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/bf4623ff36614220aa1e82b647f84990
🧪 View experiment at: https://dagshub.com/e

🏃 View run big-foal-530 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/b980c5abd49c41ce9dc3529e551fd157
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run defiant-auk-482 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/1707741c355040c6989bf031ced3aabf
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run trusting-wren-927 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/cbb2eba8179c47ab85221f5dec905df0
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run burly-lamb-360 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/ca08dd4e3c9944339b21c2c15a8994fc
🧪 View experiment at: https://dagshub.com/es

### Fixação do resultado pois deu erro no Log do MLFlow

In [ ]:
# Início do experimento com MLflow para registrar o modelo final
with mlflow.start_run(run_name="Best_Stacking_Model_Final"):

    # Define os melhores parâmetros encontrados na busca
    best_params = {
        'lgbm__num_leaves': 63,
        'lgbm__n_estimators': 900,
        'lgbm__learning_rate': 0.2,
        'final_estimator__C': 0.1,
        'catboost__learning_rate': 0.01,
        'catboost__l2_leaf_reg': 5,
        'catboost__iterations': 100,
        'catboost__depth': 10
    }

    # Instancia os modelos base e o meta-modelo com os parâmetros otimizados
    lgbm_params = {k.replace('lgbm__', ''): v for k, v in best_params.items() if 'lgbm__' in k}
    catboost_params = {k.replace('catboost__', ''): v for k, v in best_params.items() if 'catboost__' in k}
    final_estimator_params = {k.replace('final_estimator__', ''): v for k, v in best_params.items() if 'final_estimator__' in k}

    lgbm = LGBMClassifier(**lgbm_params, random_state=42)
    catboost = CatBoostClassifier(**catboost_params, verbose=0, random_state=42)
    final_estimator = LogisticRegression(**final_estimator_params)

    # Define o modelo de Stacking final
    final_stacking_model = StackingClassifier(
        estimators=[("lgbm", lgbm), ("catboost", catboost)],
        final_estimator=final_estimator,
        cv=5,
        n_jobs=-1
    )

    # Treinamento do modelo final no conjunto de treino completo e escalonado
    final_stacking_model.fit(X_train_scaled, y_train)

    # Log dos melhores hiperparâmetros
    for param, value in best_params.items():
        mlflow.log_param(param, value)

    # Avaliação e log via sua função customizada
    evaluate_and_log_model(
        "stacking",
        "Modelo_Final_Producao",
        final_stacking_model,
        X_test_scaled,
        y_test
    )

2025/08/28 11:08:52 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


=== Avaliação do Modelo: Modelo_Final_Producao ===
              precision    recall  f1-score   support

           0      0.791     0.764     0.778      8805
           1      0.778     0.824     0.800     15873
           2      0.769     0.676     0.720      5322

    accuracy                          0.780     30000
   macro avg      0.779     0.755     0.766     30000
weighted avg      0.780     0.780     0.779     30000

Recall da classe 0 (Poor): 0.764
🏃 View run Best_Stacking_Model_Final at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/55b2003a07654c12b99f81fe50641f8a
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


Resultado equilibrado: acc=0,780, recall_macro=0,755, F1_macro=0,766.
Foco de negócio: recall da classe 0 (Poor)=0,764 — razoável, porém ainda com espaço para ganho.



## Experimentação para diminuir o overfitting no modelo em produção

###  1 - Stacking com pré-ajuste via early stopping para reduzir overfitting (LGBM + CatBoost → RF meta)

Esta cédula combate overfitting ajustando o número de iterações dos base learners por CV estratificada (k=5) com early stopping (paciência=50).
Para o LGBM, busca-se best_iteration_ usando multi_logloss; para CatBoost, get_best_iteration() com TotalF1.
Em cada fold, treinamos em treino e validamos no holdout do fold; a função suporta pandas/NumPy sem fricção.
Agregamos os melhores por fold tomando a mediana (robusta) e recriamos os modelos com essas iterações “ótimas”.
O empilhamento usa StackingClassifier com passthrough=False e RandomForest raso (max_depth=3) como meta-learner.
Objetivo: reduzir a capacidade efetiva (menos árvores úteis), abaixar a acurácia de treino e fechar o gap treino–teste.

In [ ]:
def _best_iters_lgbm(X, y, base_params, n_splits=5, patience=50, random_state=42):
    """Descobre melhores n_estimators (best_iteration_) para LGBM via CV com early stopping.
    Funciona com pandas DataFrame/Series OU NumPy arrays.
    """
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    best_iters = []

    for tr_idx, va_idx in skf.split(X, y):
        # Suporta pandas ou NumPy
        if hasattr(X, "iloc"):
            X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        else:
            X_tr, X_va = X[tr_idx], X[va_idx]

        if hasattr(y, "iloc"):
            y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]
        else:
            y_tr, y_va = y[tr_idx], y[va_idx]

        lgbm = LGBMClassifier(**base_params)
        lgbm.fit(
            X_tr, y_tr,
            eval_set=[(X_va, y_va)],
            eval_metric="multi_logloss",
            callbacks=[lgb.early_stopping(stopping_rounds=patience, verbose=False)]
        )

        it = getattr(lgbm, "best_iteration_", None)
        best_iters.append(int(it if it is not None else base_params.get("n_estimators", 500)))

    return int(np.median(best_iters))


def _best_iters_catboost(X, y, base_params, n_splits=5, patience=50, random_state=42):
    """Descobre melhores iterations (get_best_iteration) para CatBoost via CV com early stopping.
    Funciona com pandas DataFrame/Series OU NumPy arrays.
    """
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    best_iters = []

    for tr_idx, va_idx in skf.split(X, y):
        # Suporta pandas ou NumPy
        if hasattr(X, "iloc"):
            X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        else:
            X_tr, X_va = X[tr_idx], X[va_idx]

        if hasattr(y, "iloc"):
            y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]
        else:
            y_tr, y_va = y[tr_idx], y[va_idx]

        cb = CatBoostClassifier(**base_params)
        cb.fit(
            X_tr, y_tr,
            eval_set=(X_va, y_va),
            verbose=False,
            early_stopping_rounds=patience,
            use_best_model=True
        )

        it = cb.get_best_iteration()
        best_iters.append(int(it if it is not None and it > 0 else base_params.get("iterations", 750)))

    return int(np.median(best_iters))


with mlflow.start_run(run_name="Stacking_LGBM_CatBoost_RFMeta_EarlyStopTuned"):

    # ---------------------------
    # 1) Pré-ajuste com early stopping (não mexe no Stacking ainda)
    # ---------------------------
    lgbm_base_params = dict(
        n_estimators=500,
        learning_rate=0.1,
        num_leaves=127,
        max_depth=-1,
        subsample=0.7,
        colsample_bytree=1.0,
        objective='multiclass',
        num_class=3,
        random_state=42
    )

    cat_base_params = dict(
        iterations=750,
        learning_rate=0.05,
        depth=10,
        l2_leaf_reg=3,
        border_count=64,
        loss_function='MultiClass',
        eval_metric='TotalF1',
        verbose=0,
        random_state=42
    )

    # Descobrir melhores iterações por CV com early stopping
    best_lgbm_iters = _best_iters_lgbm(
        X_train, y_train,
        base_params=lgbm_base_params,
        n_splits=5,
        patience=50,
        random_state=42
    )
    best_cat_iters = _best_iters_catboost(
        X_train, y_train,
        base_params=cat_base_params,
        n_splits=5,
        patience=50,
        random_state=42
    )

    # Log dos melhores valores encontrados
    mlflow.log_param("early_stop_search_cv", 5)
    mlflow.log_param("early_stop_patience", 50)
    mlflow.log_param("best_lgbm_n_estimators", best_lgbm_iters)
    mlflow.log_param("best_catboost_iterations", best_cat_iters)

    # ---------------------------
    # 2) Recriar base learners com iterações reduzidas
    # ---------------------------
    lgbm_final = LGBMClassifier(
        **{**lgbm_base_params, "n_estimators": best_lgbm_iters}
    )

    catboost_final = CatBoostClassifier(
        **{**cat_base_params, "iterations": best_cat_iters}
    )

    estimators = [
        ("lgbm", lgbm_final),
        ("catboost", catboost_final)
    ]

    # ---------------------------
    # 3) Meta-modelo e Stacking
    # ---------------------------
    final_estimator = RandomForestClassifier(
        n_estimators=100,
        max_depth=3,
        random_state=42
    )

    stacking_clf = StackingClassifier(
        estimators=estimators,
        final_estimator=final_estimator,
        cv=5,
        n_jobs=-1,
        passthrough=False
    )

    # Treinamento final do ensemble
    stacking_clf.fit(X_train, y_train)

    # ---------------------------
    # 4) Avaliação + MLflow
    # ---------------------------
    evaluate_and_log_model(
        "stacking",
        "Stacking LGBM+CatBoost (early-stop tuned) → RF meta",
        stacking_clf,
        X_test, y_test
    )

    # Logs extras
    mlflow.log_param("ensemble_method", "stacking")
    mlflow.log_param("meta_estimator", "random_forest")
    mlflow.log_param("passthrough", False)
    mlflow.log_param("cv", 5)
    mlflow.log_param("lgbm_learning_rate", lgbm_base_params["learning_rate"])
    mlflow.log_param("lgbm_num_leaves", lgbm_base_params["num_leaves"])
    mlflow.log_param("catboost_depth", cat_base_params["depth"])
    mlflow.log_param("catboost_l2_leaf_reg", cat_base_params["l2_leaf_reg"])


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005810 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2704
[LightGBM] [Info] Number of data points in the train set: 56000, number of used features: 48
[LightGBM] [Info] Start training from score -1.243184
[LightGBM] [Info] Start training from score -0.629468
[LightGBM] [Info] Start training from score -1.722267
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008432 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2707
[LightGBM] [Info] Number of data points in the train set: 56000, number of used features: 48
[LightGBM] [Info] Start training from score -1.243184
[LightGBM] [Info] Start training from score -0.629468
[LightGBM] [Info] 

2025/08/28 12:08:24 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


=== Avaliação do Modelo: Stacking LGBM+CatBoost (early-stop tuned) → RF meta ===
              precision    recall  f1-score   support

           0      0.770     0.814     0.791      8805
           1      0.815     0.780     0.797     15873
           2      0.722     0.748     0.735      5322

    accuracy                          0.784     30000
   macro avg      0.769     0.780     0.774     30000
weighted avg      0.785     0.784     0.784     30000

Recall da classe 0 (Poor): 0.814
🏃 View run Stacking_LGBM_CatBoost_RFMeta_EarlyStopTuned at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/a2bf38a68ba3447882634914df519bf9
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


### 2


#### Stacking 2A — LGBM regularizado + early stopping retunado (CatBoost reutilizado) → RF meta

Nesta cédula reforçamos o combate ao overfitting atuando no LGBM: reduzimos capacidade e adicionamos regularização explícita (num_leaves=95, max_depth=10, min_child_samples=60, reg_lambda=1.0, reg_alpha=0.1, min_split_gain=0.1, colsample_bytree=0.8, subsample=0.7, subsample_freq=1).
O n_estimators do LGBM é recalibrado por CV (k=5) com early stopping (paciência=40), usando multi_logloss; adotamos a mediana dos best_iteration_ como valor final, visando estabilidade.
O CatBoost é reaproveitado com as iterações já validadas (ou fallback 750), mantendo eval_metric=TotalF1 e learning_rate=0.05.
Construímos o ensemble via StackingClassifier com passthrough=False, meta-learner RandomForest raso (max_depth=3), e CV=5 interna para as predições-meta.
Objetivo: diminuir capacidade efetiva do LGBM e fechar o gap treino–teste sem alterar arquitetura, preservando recall por classe (especialmente classe 0 – Poor).

In [ ]:
def _best_iters_lgbm(X, y, base_params, n_splits=5, patience=40, random_state=42):
    """Estimativa do melhor n_estimators para LGBM via CV com early stopping.
    Funciona com pandas ou NumPy.
    """
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    best_iters = []
    for tr_idx, va_idx in skf.split(X, y):
        if hasattr(X, "iloc"):
            X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        else:
            X_tr, X_va = X[tr_idx], X[va_idx]

        if hasattr(y, "iloc"):
            y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]
        else:
            y_tr, y_va = y[tr_idx], y[va_idx]

        mdl = LGBMClassifier(**base_params)
        mdl.fit(
            X_tr, y_tr,
            eval_set=[(X_va, y_va)],
            eval_metric="multi_logloss",
            callbacks=[lgb.early_stopping(stopping_rounds=patience, verbose=False)]
        )
        it = getattr(mdl, "best_iteration_", None)
        best_iters.append(int(it if it is not None else base_params.get("n_estimators", 500)))
    return int(np.median(best_iters))


with mlflow.start_run(run_name="Stacking_LGBM_CatBoost_RFMeta_LGBMReg_2A-Repeat"):
    # ---------------------------
    # 1) LGBM com regularização leve (apertos sugeridos)
    # ---------------------------
    lgbm_reg_params = dict(
        # manter learning_rate e subsample base
        learning_rate=0.1,
        subsample=0.7,
        subsample_freq=1,         # ativa bagging
        colsample_bytree=0.8,     # feature_fraction
        # capacidade/complexidade
        num_leaves=95,            # ↓ de 127
        max_depth=10,             # antes -1
        min_child_samples=60,     # ↑ de 20
        # regularização explícita
        reg_lambda=1.0,           # L2
        reg_alpha=0.1,            # L1
        min_split_gain=0.1,
        # demais
        objective="multiclass",
        num_class=3,
        random_state=42,
        n_estimators=600          # valor provisório; será recalibrado via early stopping
    )

    # 2) Re-otimizar n_estimators do LGBM com ES (CV=5, patience=40)
    best_lgbm_iters_2a = _best_iters_lgbm(
        X_train, y_train,
        base_params=lgbm_reg_params,
        n_splits=5,
        patience=40,
        random_state=42
    )

    mlflow.log_param("stage", "2A_LGBM_regularized")
    mlflow.log_param("early_stop_search_cv", 5)
    mlflow.log_param("early_stop_patience", 40)
    mlflow.log_param("best_lgbm_n_estimators_2A", best_lgbm_iters_2a)

    # ---------------------------
    # 3) CatBoost reaproveitado (fallback para 750 se não foi definido antes)
    # ---------------------------
    try:
        best_cat_iters  # se você definiu antes (ex.: best_cat_iters = 6XX)
    except NameError:
        best_cat_iters = 750  # fallback seguro

    cat_params = dict(
        iterations=best_cat_iters,
        learning_rate=0.05,
        depth=10,
        l2_leaf_reg=3,
        border_count=64,
        loss_function="MultiClass",
        eval_metric="TotalF1",
        verbose=0,
        random_state=42
    )
    mlflow.log_param("catboost_iterations_used", best_cat_iters)

    # ---------------------------
    # 4) Montar ensemble e treinar
    # ---------------------------
    lgbm_final_2a = LGBMClassifier(**{**lgbm_reg_params, "n_estimators": best_lgbm_iters_2a})
    catboost_final = CatBoostClassifier(**cat_params)

    estimators = [
        ("lgbm", lgbm_final_2a),
        ("catboost", catboost_final)
    ]

    final_estimator = RandomForestClassifier(
        n_estimators=100,
        max_depth=3,
        random_state=42
    )

    stacking_clf_2a = StackingClassifier(
        estimators=estimators,
        final_estimator=final_estimator,
        cv=5,
        n_jobs=-1,
        passthrough=False
    )

    stacking_clf_2a.fit(X_train, y_train)

    # ---------------------------
    # 5) Avaliação + MLflow
    # ---------------------------
    evaluate_and_log_model(
        "stacking",
        "Stacking LGBM+CatBoost (LGBM regularizado + ES retuned) → RF meta",
        stacking_clf_2a,
        X_test, y_test
    )

    # Log dos hiperparâmetros chave do LGBM (para rastreio fácil)
    mlflow.log_param("lgbm_num_leaves_2A", lgbm_reg_params["num_leaves"])
    mlflow.log_param("lgbm_max_depth_2A", lgbm_reg_params["max_depth"])
    mlflow.log_param("lgbm_min_child_samples_2A", lgbm_reg_params["min_child_samples"])
    mlflow.log_param("lgbm_reg_lambda_2A", lgbm_reg_params["reg_lambda"])
    mlflow.log_param("lgbm_reg_alpha_2A", lgbm_reg_params["reg_alpha"])
    mlflow.log_param("lgbm_min_split_gain_2A", lgbm_reg_params["min_split_gain"])
    mlflow.log_param("lgbm_colsample_bytree_2A", lgbm_reg_params["colsample_bytree"])
    mlflow.log_param("lgbm_subsample_2A", lgbm_reg_params["subsample"])
    mlflow.log_param("lgbm_subsample_freq_2A", lgbm_reg_params["subsample_freq"])


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005338 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2704
[LightGBM] [Info] Number of data points in the train set: 56000, number of used features: 48
[LightGBM] [Info] Start training from score -1.243184
[LightGBM] [Info] Start training from score -0.629468
[LightGBM] [Info] Start training from score -1.722267
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further 

2025/09/14 22:01:47 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


=== Avaliação do Modelo: Stacking LGBM+CatBoost (LGBM regularizado + ES retuned) → RF meta ===
              precision    recall  f1-score   support

           0      0.764     0.819     0.791      8805
           1      0.819     0.777     0.797     15873
           2      0.727     0.753     0.740      5322

    accuracy                          0.785     30000
   macro avg      0.770     0.783     0.776     30000
weighted avg      0.787     0.785     0.785     30000

Recall da classe 0 (Poor): 0.819
Acurácia de Treino: 0.919
🏃 View run Stacking_LGBM_CatBoost_RFMeta_LGBMReg_2A-Repeat at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/01fce9d091a5491d98d697da32e6f712
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


🏃 View run polite-rat-797 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/7cf0f2a3e19c47da9174a4321ca60d91
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run incongruous-ray-564 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/df1d544caa834158b5164b1a6ce0b2c0
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


🏃 View run powerful-gnu-951 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/10ee9a3c6e3f424cb93f8d3a82af862f
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


#### Stacking 2B — Random Search focal no LGBM (ao redor do 2A) + retune de iterações, CatBoost fixo (747) → RF meta

Após o bom baseline do 2A (LGBM regularizado), este passo busca refinar só o LGBM com um RandomizedSearchCV enxuto (n_iter=12, cv=3) em torno do espaço que funcionou no 2A (num_leaves, max_depth, min_child_samples, reg_λ/α, colsample, subsample, min_split_gain, subsample_freq).
Fixamos o CatBoost nas 747 iterações (resgatadas do MLflow) para isolar o efeito do LGBM, e incluímos scorers de recall_macro (principal) e recall_poor (secundário) na busca.
Para evitar under/overfit por contagem de árvores, reestimamos n_estimators do LGBM vencedor via CV=5 com early stopping (paciência=35), usando a mediana do best_iteration como valor final estável.
O ensemble final mantém o StackingClassifier com passthrough=False e RandomForest raso (max_depth=3) como meta-learner, preservando a generalização do nível meta.
Tudo é rastreado no MLflow: hiperparâmetros do top-1, métricas da busca, iterações refinalizadas e avaliação de holdout, permitindo comparação direta com o 2A (inclusive foco em Recall_Poor).
Estratégia: apertar o LGBM no ponto ótimo local com orçamento computacional contido, mantendo o CatBoost estável para medir o ganho incremental real no stack.

In [ ]:
# CatBoost: iterações fixadas a partir do MLflow (run "Stacking_LGBM_CatBoost_RFMeta_EarlyStopTuned" -> param "best_catboost_iterations" = 747)
best_cat_iters = 747
assert isinstance(best_cat_iters, int) and best_cat_iters > 0

def _best_iters_lgbm(X, y, base_params, n_splits=5, patience=35, random_state=42):
    """Reestima o melhor n_estimators via CV com early stopping (suporta pandas/NumPy)."""
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    best_iters = []
    for tr_idx, va_idx in skf.split(X, y):
        X_tr, X_va = (X.iloc[tr_idx], X.iloc[va_idx]) if hasattr(X, "iloc") else (X[tr_idx], X[va_idx])
        y_tr, y_va = (y.iloc[tr_idx], y.iloc[va_idx]) if hasattr(y, "iloc") else (y[tr_idx], y[va_idx])

        mdl = LGBMClassifier(**base_params)
        mdl.fit(
            X_tr, y_tr,
            eval_set=[(X_va, y_va)],
            eval_metric="multi_logloss",
            callbacks=[lgb.early_stopping(stopping_rounds=patience, verbose=False)]
        )
        it = getattr(mdl, "best_iteration_", None)
        best_iters.append(int(it if it is not None else base_params.get("n_estimators", 350)))
    return int(np.median(best_iters))

def _recall_poor(y_true, y_pred):
    """Recall da classe 0 (Poor) para monitoramento secundário na busca."""
    per_class = recall_score(y_true, y_pred, labels=[0, 1, 2], average=None, zero_division=0)
    return float(per_class[0])

recall_macro_scorer = make_scorer(recall_score, average="macro", zero_division=0)
recall_poor_scorer  = make_scorer(_recall_poor)

with mlflow.start_run(run_name="Stacking_LGBM_CatBoost_RFMeta_LGBMRandomSearch_2B_PlanA"):
    # Metadados e rastreabilidade
    mlflow.log_param("stage", "2B_LGBM_random_search_PlanA")
    mlflow.log_param("budget_n_iter", 12)
    mlflow.log_param("budget_cv_search", 3)
    mlflow.log_param("budget_cv_es", 5)
    mlflow.set_tag("catboost_iters_source", "MLflow: Stacking_LGBM_CatBoost_RFMeta_EarlyStopTuned::best_catboost_iterations=747")
    mlflow.log_param("catboost_iterations_used", best_cat_iters)

    # Base do LGBM na busca (sem early stopping; n_estimators moderado para acelerar)
    lgbm_base_fixed = dict(
        learning_rate=0.1,
        objective="multiclass",
        num_class=3,
        random_state=42,
        n_estimators=350,   # menor que 500 para reduzir custo por fit; será reotimizado depois
        verbosity=-1
    )

    # Espaço de busca focado ao redor do 2.A
    param_distributions = {
        "num_leaves":        [80, 85, 90, 95, 100, 105, 110],
        "max_depth":         [8, 9, 10, 11, 12],
        "min_child_samples": [40, 50, 60, 70, 80],
        "reg_lambda":        [0.5, 0.75, 1.0, 1.5, 2.0, 3.0],
        "reg_alpha":         [0.05, 0.1, 0.2, 0.3, 0.5],
        "min_split_gain":    [0.0, 0.05, 0.1, 0.15],
        "colsample_bytree":  [0.75, 0.8, 0.85, 0.9],
        "subsample":         [0.6, 0.65, 0.7, 0.75, 0.8],
        "subsample_freq":    [1, 2, 3],
    }

    # Busca enxuta: 12 amostras x CV=3 = 36 fits
    lgbm_search_model = LGBMClassifier(**lgbm_base_fixed)
    skf_search = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    search = RandomizedSearchCV(
        estimator=lgbm_search_model,
        param_distributions=param_distributions,
        n_iter=12,
        scoring={"recall_macro": recall_macro_scorer, "recall_poor": recall_poor_scorer},
        refit=False,               # evita 1 fit extra
        cv=skf_search,
        n_jobs=-1,
        verbose=1,
        random_state=42,
        return_train_score=False
    )
    search.fit(X_train, y_train)

    # Seleção manual do melhor por recall_macro e logging dos resultados
    results_df = pd.DataFrame(search.cv_results_).copy()
    results_df.sort_values("mean_test_recall_macro", ascending=False, inplace=True)
    best_idx = int(np.argmax(search.cv_results_["mean_test_recall_macro"]))
    best_recall_macro = float(search.cv_results_["mean_test_recall_macro"][best_idx])
    best_recall_poor  = float(search.cv_results_["mean_test_recall_poor"][best_idx])
    mlflow.log_metric("best_cv_recall_macro_2B", best_recall_macro)
    mlflow.log_metric("best_cv_recall_poor_2B", best_recall_poor)

    # Log dos melhores hiperparâmetros da busca
    best_params_search = results_df.iloc[0]["params"]
    for k, v in best_params_search.items():
        mlflow.log_param(f"lgbm_best_{k}_2B", v)

    # Artefatos: resultados completos e top-5
    with tempfile.TemporaryDirectory() as tmpdir:
        all_path  = os.path.join(tmpdir, "lgbm_random_search_results_full.csv")
        top5_path = os.path.join(tmpdir, "lgbm_random_search_top5.csv")
        results_df.to_csv(all_path, index=False)
        results_df.head(5).to_csv(top5_path, index=False)
        mlflow.log_artifact(all_path,  artifact_path="random_search")
        mlflow.log_artifact(top5_path, artifact_path="random_search")

    # Retune de n_estimators do LGBM com early stopping (CV=5; ~5 fits)
    best_params = {**lgbm_base_fixed, **best_params_search, "n_estimators": 500}  # provisório para ES
    best_lgbm_iters_2B = _best_iters_lgbm(
        X_train, y_train,
        base_params=best_params,
        n_splits=5,
        patience=35,
        random_state=42
    )
    mlflow.log_param("best_lgbm_n_estimators_2B", int(best_lgbm_iters_2B))

    # Ensemble final: LGBM vencedor + CatBoost fixo + RF meta (Stacking CV=5; ~13 fits)
    lgbm_final_2b = LGBMClassifier(**{**best_params, "n_estimators": int(best_lgbm_iters_2B)})
    catboost_final = CatBoostClassifier(
        iterations=best_cat_iters,
        learning_rate=0.05,
        depth=10,
        l2_leaf_reg=3,
        border_count=64,
        loss_function="MultiClass",
        eval_metric="TotalF1",
        verbose=0,
        random_state=42
    )
    estimators = [("lgbm", lgbm_final_2b), ("catboost", catboost_final)]
    final_estimator = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=42)

    stacking_clf_2b = StackingClassifier(
        estimators=estimators,
        final_estimator=final_estimator,
        cv=5,
        n_jobs=-1,
        passthrough=False
    )
    stacking_clf_2b.fit(X_train, y_train)

    # Avaliação e logging no MLflow
    evaluate_and_log_model(
        "stacking",
        "Stacking LGBM+CatBoost (LGBM RandomSearch PlanA + ES retuned) → RF meta",
        stacking_clf_2b,
        X_test, y_test
    )

    # Logging de parâmetros-chave do LGBM (para rastreabilidade)
    for hp in ["num_leaves", "max_depth", "min_child_samples", "reg_lambda", "reg_alpha",
               "min_split_gain", "colsample_bytree", "subsample", "subsample_freq"]:
        mlflow.log_param(f"lgbm_{hp}_2B_final", getattr(lgbm_final_2b, hp))

2025/08/28 18:51:42 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Fitting 3 folds for each of 12 candidates, totalling 36 fits


2025/08/28 18:58:51 WARNING mlflow.sklearn.utils: Failed to autolog metrics for RandomizedSearchCV. Logging error: This 'RandomizedSearchCV' has no attribute 'predict'
2025/08/28 18:58:51 WARNING mlflow.sklearn.utils: _get_classifier_artifacts.<locals>.plot_confusion_matrix failed. The artifact training_confusion_matrix will not be recorded. Artifact error: This 'RandomizedSearchCV' has no attribute 'predict'
2025/08/28 18:59:10 WARNING mlflow.sklearn.utils: BaseSearchCV.score failed. The 'training_score' metric will not be recorded. Scoring error: This RandomizedSearchCV instance was initialized with `refit=False`. score is available only after refitting on the best parameters. You can refit an estimator manually using the `best_params_` attribute
2025/08/28 18:59:11 WARNING mlflow.sklearn: Failed to infer model signature: the trained model does not have a `predict` or `transform` function, which is required in order to infer the signature
2025/08/28 18:59:11 WARNING mlflow.sklearn: M

=== Avaliação do Modelo: Stacking LGBM+CatBoost (LGBM RandomSearch PlanA + ES retuned) → RF meta ===
              precision    recall  f1-score   support

           0      0.768     0.821     0.793      8805
           1      0.821     0.779     0.799     15873
           2      0.730     0.759     0.744      5322

    accuracy                          0.788     30000
   macro avg      0.773     0.786     0.779     30000
weighted avg      0.789     0.788     0.788     30000

Recall da classe 0 (Poor): 0.821
Acurácia de Treino: 0.938
🏃 View run Stacking_LGBM_CatBoost_RFMeta_LGBMRandomSearch_2B_PlanA at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/74733e4ed7c94616ad33400c8c834bc5
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


Resultado global melhorou levemente: acc=0,788, recall_macro=0,786, F1_macro=0,779 (vs. 2A: 0,785/0,783/0,776).
Classe 0 (Poor) subiu para recall=0,821 (↑ +0,2 p.p. vs. 2A), mantendo o foco de negócio.
Classe 2 (Good) também melhora moderada (recall=0,759 vs. 0,753), indicando ganho de equilíbrio.
Classe 1 estável (recall=0,779 vs. 0,777), sem regressão relevante.
Sinal de overfitting aumentou: train acc=0,938 vs. test acc=0,788 (gap ~15,0 p.p.; em 2A era ~13,4 p.p.).
Leitura: o Random Search em torno do 2A encontrou um ponto ligeiramente superior em holdout, ao custo de um pouco mais de ajuste no treino.

### Stacking 3A — CatBoost regularizado + early stopping (LGBM 2A fixo) → RF meta

Nesta cédula, fixamos o LGBM do 2A e atuamos apenas no CatBoost para reduzir overfitting, mantendo foco em recall da classe 0 (Poor).
O CatBoost recebe apertos de regularização: depth 10→8, l2_leaf_reg 3→8, rsm=0.85, subsample=0.8 com bootstrap_type='Bernoulli', random_strength=1.5, border_count=64, learning_rate=0.05.
Estimamos o nº ótimo de iterações via CV estratificada k=5 com early stopping (patience=40), tomando a mediana do get_best_iteration() para estabilidade.
O LGBM permanece com o pacote regularizado do 2A (num_leaves=95, max_depth=10, min_child_samples=60, reg_lambda=1.0, reg_alpha=0.1, min_split_gain=0.1, colsample_bytree=0.8, subsample=0.7).
O ensemble é um StackingClassifier com passthrough=False e RandomForest raso (n_estimators=100, max_depth=3) como meta-learner, visando generalização no nível meta.
Objetivo: reduzir a capacidade/variância do CatBoost para diminuir a acurácia de treino e fechar o gap treino–teste sem sacrificar recall_macro e Recall_Poor.

In [ ]:
# === Passo 3A: CatBoost regularizado (aperto leve) + ES para melhor iterations | LGBM fixo do 2.A ===
# Objetivo: reduzir overfit no base CatBoost mantendo Recall Poor alto, reaproveitando o LGBM do 2.A.


def _best_iters_catboost(X, y, base_params, n_splits=5, patience=40, random_state=42):
    """Estimativa de iterations ótima para CatBoost via CV com early stopping (suporta pandas/NumPy)."""
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    best_iters = []
    for tr_idx, va_idx in skf.split(X, y):
        X_tr, X_va = (X.iloc[tr_idx], X.iloc[va_idx]) if hasattr(X, "iloc") else (X[tr_idx], X[va_idx])
        y_tr, y_va = (y.iloc[tr_idx], y.iloc[va_idx]) if hasattr(y, "iloc") else (y[tr_idx], y[va_idx])

        cb = CatBoostClassifier(**base_params)
        cb.fit(
            X_tr, y_tr,
            eval_set=(X_va, y_va),
            verbose=False,
            early_stopping_rounds=patience,
            use_best_model=True
        )
        it = cb.get_best_iteration()
        best_iters.append(int(it if it is not None and it > 0 else base_params.get("iterations", 800)))
    return int(np.median(best_iters))


with mlflow.start_run(run_name="Stacking_LGBM_CatBoost_RFMeta_CatBoostReg_3A"):
    # --- LGBM fixo (2.A) ---
    lgbm_params_2a = dict(
        learning_rate=0.1,
        colsample_bytree=0.8,
        min_split_gain=0.1,
        subsample=0.7,
        subsample_freq=1,
        num_leaves=95,
        max_depth=10,
        min_child_samples=60,
        reg_lambda=1.0,
        reg_alpha=0.1,
        objective="multiclass",
        num_class=3,
        random_state=42,
        n_estimators=408
    )

    # --- CatBoost 3A: regularização leve (aperto) ---
    # depth↓, l2_leaf_reg↑, feature sampling (rsm), subsample + bootstrap Bernoulli, leve random_strength
    cat_base_params_3a = dict(
        iterations=900,                 # upper bound; ES definirá o ótimo
        learning_rate=0.05,
        depth=8,                        # de 10 -> 8
        l2_leaf_reg=8,                  # de 3 -> 8
        rsm=0.85,                       # feature sampling
        subsample=0.8,                  # amostragem de registros
        bootstrap_type="Bernoulli",
        random_strength=1.5,
        border_count=64,
        loss_function="MultiClass",
        eval_metric="TotalF1",
        verbose=0,
        random_state=42
    )

    mlflow.log_param("stage", "3A_CatBoost_regularized")
    for k, v in {**lgbm_params_2a}.items():
        mlflow.log_param(f"lgbm_2A_{k}", v)
    for k, v in {**cat_base_params_3a}.items():
        if k != "iterations":
            mlflow.log_param(f"catboost_3A_{k}", v)

    # --- Early stopping só no CatBoost para achar melhor iterations ---
    best_cat_iters_3a = _best_iters_catboost(
        X_train, y_train,
        base_params=cat_base_params_3a,
        n_splits=5,
        patience=40,
        random_state=42
    )
    mlflow.log_param("best_catboost_iterations_3A", int(best_cat_iters_3a))

    # --- Montagem do ensemble (LGBM 2.A + CatBoost 3.A) ---
    lgbm_final = LGBMClassifier(**lgbm_params_2a)
    catboost_final = CatBoostClassifier(**{**cat_base_params_3a, "iterations": int(best_cat_iters_3a)})

    estimators = [
        ("lgbm", lgbm_final),
        ("catboost", catboost_final)
    ]

    final_estimator = RandomForestClassifier(
        n_estimators=100,
        max_depth=3,
        random_state=42
    )

    stacking_clf_3a = StackingClassifier(
        estimators=estimators,
        final_estimator=final_estimator,
        cv=5,
        n_jobs=-1,
        passthrough=False
    )

    # --- Treino e avaliação ---
    stacking_clf_3a.fit(X_train, y_train)

    evaluate_and_log_model(
        "stacking",
        "Stacking LGBM(2A) + CatBoost(3A reg + ES) → RF meta",
        stacking_clf_3a,
        X_test, y_test
    )

    # Log dos principais hiperparâmetros finais (rastreabilidade)
    mlflow.log_param("catboost_3A_iterations_final", int(best_cat_iters_3a))
    mlflow.log_param("lgbm_2A_n_estimators_final", lgbm_params_2a["n_estimators"])


2025/08/29 14:35:22 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


=== Avaliação do Modelo: Stacking LGBM(2A) + CatBoost(3A reg + ES) → RF meta ===
              precision    recall  f1-score   support

           0      0.763     0.820     0.790      8805
           1      0.823     0.771     0.796     15873
           2      0.718     0.766     0.741      5322

    accuracy                          0.784     30000
   macro avg      0.768     0.785     0.776     30000
weighted avg      0.787     0.784     0.785     30000

Recall da classe 0 (Poor): 0.820
Acurácia de Treino: 0.920
🏃 View run Stacking_LGBM_CatBoost_RFMeta_CatBoostReg_3A at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/960226a7f2844c4e80463e06a713ab72
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


### Stacking 3B-1 — CatBoost agressivo (Bernoulli + subsample) com LGBM 2A fixo → RF meta

Nesta cédula mantemos o LGBM do 2A fixo e tornamos o CatBoost mais conservador para cortar variância: depth=7 (árvores mais rasas), l2_leaf_reg=12 (L2 ↑), rsm=0.80 (amostragem de features), bootstrap_type='Bernoulli' com subsample=0.75 (amostragem de linhas), random_strength=2.5 (ruído controlado) e border_count=48 (menos bins).
Estimamos o nº ótimo de iterações do CatBoost por CV k=5 com early stopping (patience=45) e adotamos a mediana do get_best_iteration() para estabilidade entre folds.
O ensemble usa StackingClassifier com passthrough=False e RandomForest raso (n_estimators=100, max_depth=3) como meta-learner, privilegiando generalização no nível meta.
Objetivo: reduzir overfitting do CatBoost (baixar train acc/encurtar gap) sem perder o recall macro e mantendo o recall da classe 0 (Poor) elevado.


In [ ]:
# === Passo 3B-1: CatBoost mais agressivo (Bernoulli + subsample) + LGBM fixo (2.A) + meta RF ===
# Objetivo: reduzir overfit do CatBoost mantendo/ganhando Recall Poor e macro, isolando o efeito no base learner.

def _best_iters_catboost(X, y, base_params, n_splits=5, patience=45, random_state=42):
    """Reestima o melhor 'iterations' para CatBoost via CV + early stopping (compatível com pandas/NumPy)."""
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    best_iters = []
    for tr_idx, va_idx in skf.split(X, y):
        X_tr, X_va = (X.iloc[tr_idx], X.iloc[va_idx]) if hasattr(X, "iloc") else (X[tr_idx], X[va_idx])
        y_tr, y_va = (y.iloc[tr_idx], y.iloc[va_idx]) if hasattr(y, "iloc") else (y[tr_idx], y[va_idx])

        cb = CatBoostClassifier(**base_params)
        cb.fit(
            X_tr, y_tr,
            eval_set=(X_va, y_va),
            verbose=False,
            early_stopping_rounds=patience,
            use_best_model=True
        )
        it = cb.get_best_iteration()
        best_iters.append(int(it if it is not None and it > 0 else base_params.get("iterations", 1200)))
    return int(sorted(best_iters)[len(best_iters)//2])  # mediana

with mlflow.start_run(run_name="Stacking_LGBM_CatBoost_RFMeta_CatBoostReg_3B1"):
    # LGBM fixo (2.A)
    lgbm_params_2a = dict(
        learning_rate=0.1,
        colsample_bytree=0.8,
        min_split_gain=0.1,
        subsample=0.7,
        subsample_freq=1,
        num_leaves=95,
        max_depth=10,
        min_child_samples=60,
        reg_lambda=1.0,
        reg_alpha=0.1,
        objective="multiclass",
        num_class=3,
        random_state=42,
        n_estimators=408
    )

    # CatBoost 3B-1 (Bernoulli + subsample)
    cat_base_params_3b1 = dict(
        iterations=1200,              # upper bound; ES definirá o ótimo
        learning_rate=0.05,
        depth=7,                      # ↓ profundidade
        l2_leaf_reg=12,               # ↑ regularização L2
        rsm=0.80,                     # feature sampling
        bootstrap_type="Bernoulli",   # usa subsample (não usar bagging_temperature)
        subsample=0.75,               # amostragem de linhas
        random_strength=2.5,          # ruído controlado nos splits
        border_count=48,              # menos bins -> menor variância
        loss_function="MultiClass",
        eval_metric="TotalF1",
        verbose=0,
        random_state=42
    )

    # Logging de estágio e parâmetros
    mlflow.log_param("stage", "3B1_CatBoost_Bernoulli_RFMeta")
    for k, v in lgbm_params_2a.items():
        mlflow.log_param(f"lgbm_2A_{k}", v)
    for k, v in cat_base_params_3b1.items():
        if k != "iterations":
            mlflow.log_param(f"catboost_3B1_{k}", v)

    # Early stopping do CatBoost (recalibra iterations)
    best_cat_iters_3b1 = _best_iters_catboost(
        X_train, y_train,
        base_params=cat_base_params_3b1,
        n_splits=5,
        patience=45,
        random_state=42
    )
    mlflow.log_param("best_catboost_iterations_3B1", int(best_cat_iters_3b1))

    # Ensemble final: LGBM (2.A) + CatBoost (3B-1) + meta RF
    lgbm_final = LGBMClassifier(**lgbm_params_2a)
    catboost_final = CatBoostClassifier(**{**cat_base_params_3b1, "iterations": int(best_cat_iters_3b1)})

    estimators = [
        ("lgbm", lgbm_final),
        ("catboost", catboost_final)
    ]

    final_estimator = RandomForestClassifier(
        n_estimators=100,
        max_depth=3,
        random_state=42
    )

    stacking_clf_3b1 = StackingClassifier(
        estimators=estimators,
        final_estimator=final_estimator,
        cv=5,
        n_jobs=-1,
        passthrough=False
    )

    stacking_clf_3b1.fit(X_train, y_train)

    evaluate_and_log_model(
        "stacking",
        "Stacking LGBM(2A) + CatBoost(3B1 Bernoulli + ES) → RF meta",
        stacking_clf_3b1,
        X_test, y_test
    )

2025/08/29 15:15:15 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


=== Avaliação do Modelo: Stacking LGBM(2A) + CatBoost(3B1 Bernoulli + ES) → RF meta ===
              precision    recall  f1-score   support

           0      0.757     0.829     0.792      8805
           1      0.828     0.763     0.795     15873
           2      0.717     0.772     0.743      5322

    accuracy                          0.784     30000
   macro avg      0.768     0.788     0.777     30000
weighted avg      0.788     0.784     0.785     30000

Recall da classe 0 (Poor): 0.829
Acurácia de Treino: 0.925
🏃 View run Stacking_LGBM_CatBoost_RFMeta_CatBoostReg_3B1 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/bccfb26c333a41cf94facfc225cc8f2c
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


#### Stacking 3B-2 — CatBoost com Bayesian bootstrap + meta LogReg (L2), LGBM 2A fixo

Nesta cédula mantemos o LGBM do 2A fixo e tornamos o CatBoost mais regularizado usando bootstrap_type="Bayesian" com bagging_temperature=1.8, além de depth=7, l2_leaf_reg=12, rsm=0.80, border_count=48 e random_strength=2.5 para reduzir variância.
Estimamos o nº ótimo de iterações do CatBoost via CV estratificada k=5 com early stopping (paciência=45) e adotamos a mediana do get_best_iteration().
No topo, trocamos o meta-learner para LogisticRegression (L2, lbfgs), favorecendo um agregador linear, parcimonioso e estável sobre as previsões dos bases (passthrough=False).
Objetivo: diminuir overfitting sem perder recall_macro e mantendo o recall da classe 0 (Poor) competitivo, isolando o impacto do CatBoost.

In [ ]:
# === Passo 3B-2: CatBoost mais agressivo (Bayesian bootstrap) + meta LogisticRegression (L2) ===
# Objetivo: reduzir overfit mantendo/ganhando Recall Poor e macro; LGBM fixo (2.A), CatBoost mais regularizado.
# Orçamento aproximado: ~18 fits (ES CatBoost CV=5 ≈ 5 + Stacking CV=5 ≈ 13).


def _best_iters_catboost(X, y, base_params, n_splits=5, patience=45, random_state=42):
    """Estimativa de iterations ótima para CatBoost via CV com early stopping (compatível com pandas/NumPy)."""
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    best_iters = []
    for tr_idx, va_idx in skf.split(X, y):
        X_tr, X_va = (X.iloc[tr_idx], X.iloc[va_idx]) if hasattr(X, "iloc") else (X[tr_idx], X[va_idx])
        y_tr, y_va = (y.iloc[tr_idx], y.iloc[va_idx]) if hasattr(y, "iloc") else (y[tr_idx], y[va_idx])

        cb = CatBoostClassifier(**base_params)
        cb.fit(
            X_tr, y_tr,
            eval_set=(X_va, y_va),
            verbose=False,
            early_stopping_rounds=patience,
            use_best_model=True
        )
        it = cb.get_best_iteration()
        best_iters.append(int(it if it is not None and it > 0 else base_params.get("iterations", 1200)))
    # Mediana para robustez entre folds
    return int(sorted(best_iters)[len(best_iters)//2])

with mlflow.start_run(run_name="Stacking_LGBM_CatBoost_LogRegMeta_CatBoostReg_3B2"):
    # --- LGBM fixo (2.A) ---
    lgbm_params_2a = dict(
        learning_rate=0.1,
        colsample_bytree=0.8,
        min_split_gain=0.1,
        subsample=0.7,
        subsample_freq=1,
        num_leaves=95,
        max_depth=10,
        min_child_samples=60,
        reg_lambda=1.0,
        reg_alpha=0.1,
        objective="multiclass",
        num_class=3,
        random_state=42,
        n_estimators=408
    )

    # --- CatBoost 3B-2: Bayesian bootstrap (sem subsample), mais regularização ---
    cat_base_params_3b2 = dict(
        iterations=1200,              # upper bound; ES definirá o ótimo
        learning_rate=0.05,
        depth=7,                      # ↓ profundidade
        l2_leaf_reg=12,               # ↑ regularização L2
        rsm=0.80,                     # feature sampling
        bootstrap_type="Bayesian",    # usa bagging_temperature (sem subsample)
        bagging_temperature=1.8,
        random_strength=2.5,
        border_count=48,              # menos bins -> menor variância
        loss_function="MultiClass",
        eval_metric="TotalF1",
        verbose=0,
        random_state=42
    )

    # --- Meta-estimador: LogisticRegression (L2) no topo do stacking ---
    meta_lr_params = dict(
        penalty="l2",
        C=1.0,
        solver="lbfgs",
        max_iter=500,
        multi_class="auto",
        n_jobs=None
    )

    # --- Logging de estágio e parâmetros base ---
    mlflow.log_param("stage", "3B2_CatBoost_Bayesian_LogRegMeta")
    for k, v in lgbm_params_2a.items():
        mlflow.log_param(f"lgbm_2A_{k}", v)
    for k, v in cat_base_params_3b2.items():
        if k != "iterations":
            mlflow.log_param(f"catboost_3B2_{k}", v)
    for k, v in meta_lr_params.items():
        mlflow.log_param(f"meta_lr_{k}", v)

    # --- Early stopping do CatBoost (recalibra iterations) ---
    best_cat_iters_3b2 = _best_iters_catboost(
        X_train, y_train,
        base_params=cat_base_params_3b2,
        n_splits=5,
        patience=45,
        random_state=42
    )
    mlflow.log_param("best_catboost_iterations_3B2", int(best_cat_iters_3b2))

    # --- Montagem do ensemble (LGBM 2.A + CatBoost 3B-2) ---
    lgbm_final = LGBMClassifier(**lgbm_params_2a)
    catboost_final = CatBoostClassifier(**{**cat_base_params_3b2, "iterations": int(best_cat_iters_3b2)})

    estimators = [
        ("lgbm", lgbm_final),
        ("catboost", catboost_final)
    ]

    final_estimator = LogisticRegression(**meta_lr_params)

    stacking_clf_3b2 = StackingClassifier(
        estimators=estimators,
        final_estimator=final_estimator,
        cv=5,
        n_jobs=-1,
        passthrough=False
    )

    # --- Treino e avaliação ---
    stacking_clf_3b2.fit(X_train, y_train)

    evaluate_and_log_model(
        "stacking",
        "Stacking LGBM(2A) + CatBoost(3B2 Bayesian + ES) → Meta LR(L2)",
        stacking_clf_3b2,
        X_test, y_test
    )

2025/08/29 15:00:01 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


=== Avaliação do Modelo: Stacking LGBM(2A) + CatBoost(3B2 Bayesian + ES) → Meta LR(L2) ===
              precision    recall  f1-score   support

           0      0.785     0.758     0.771      8805
           1      0.774     0.821     0.797     15873
           2      0.770     0.672     0.718      5322

    accuracy                          0.776     30000
   macro avg      0.776     0.751     0.762     30000
weighted avg      0.776     0.776     0.775     30000

Recall da classe 0 (Poor): 0.758
Acurácia de Treino: 0.949
🏃 View run Stacking_LGBM_CatBoost_LogRegMeta_CatBoostReg_3B2 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/39e4123ca6bc4f7a9380b94577015a65
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


In [23]:
# ==============================================================================
# === FUNÇÃO HELPER PARA OTIMIZAÇÃO DE ITERAÇÕES ================================
# ==============================================================================

def _best_iters_catboost(X, y, base_params, n_splits=5, patience=45, random_state=42):
    """Reestima o melhor 'iterations' para CatBoost via CV + early stopping."""
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    best_iters = []
    for tr_idx, va_idx in skf.split(X, y):
        X_tr, X_va = (X.iloc[tr_idx], X.iloc[va_idx]) if hasattr(X, "iloc") else (X[tr_idx], X[va_idx])
        y_tr, y_va = (y.iloc[tr_idx], y.iloc[va_idx]) if hasattr(y, "iloc") else (y[tr_idx], y[va_idx])

        cb = CatBoostClassifier(**base_params)
        cb.fit(
            X_tr, y_tr,
            eval_set=(X_va, y_va),
            verbose=False,
            early_stopping_rounds=patience,
            use_best_model=True
        )
        it = cb.get_best_iteration()
        best_iters.append(int(it if it is not None and it > 0 else base_params.get("iterations", 1200)))
    # Retorna a mediana para um resultado mais robusto
    return int(sorted(best_iters)[len(best_iters)//2])

# ==============================================================================
# === PIPELINE DE STACKING =====================================================
# ==============================================================================

RUN_NAME = "Stacking_LGBM_CatBoostH1_RFMeta"
RANDOM_STATE = 42

with mlflow.start_run(run_name=RUN_NAME):

    # --- 1. Definição dos Parâmetros dos Modelos Base ---

    # LGBM fixo (parâmetros do modelo 2.A do script de referência)
    lgbm_params_2a = dict(
        learning_rate=0.1,
        colsample_bytree=0.8,
        min_split_gain=0.1,
        subsample=0.7,
        subsample_freq=1,
        num_leaves=95,
        max_depth=10,
        min_child_samples=60,
        reg_lambda=1.0,
        reg_alpha=0.1,
        objective="multiclass",
        num_class=3,
        random_state=RANDOM_STATE,
        n_estimators=408  # Valor fixo do script de referência
    )

    # CatBoost com os parâmetros que você solicitou (do seu modelo H1)
    cat_params_h1 = dict(
        loss_function='MultiClass',
        eval_metric='TotalF1',
        depth=8,
        l2_leaf_reg=10.0,
        rsm=0.8,
        bagging_temperature=0.5,
        random_strength=1.8,
        border_count=32,
        learning_rate=0.05,
        random_state=RANDOM_STATE,
        verbose=0,
        thread_count=-1,
        # Adicionado para compatibilidade com a lógica de ES
        bootstrap_type='Bayesian' # Assumido, pois usa bagging_temperature
    )

    # --- 2. Otimização e Logging ---

    mlflow.log_param("stage", "Stacking_LGBM_2A_vs_CatBoost_H1_RFMeta")
    mlflow.log_params({f"lgbm_2A_{k}": v for k, v in lgbm_params_2a.items()})
    mlflow.log_params({f"catboost_H1_{k}": v for k, v in cat_params_h1.items()})

    # Early stopping do CatBoost para encontrar as iterações ótimas
    cat_params_for_es = cat_params_h1.copy()
    cat_params_for_es['iterations'] = 2000 # Teto alto para o ES
    best_cat_iters_h1 = _best_iters_catboost(
        X_train, y_train,
        base_params=cat_params_for_es,
        n_splits=5,
        patience=45,
        random_state=RANDOM_STATE
    )
    mlflow.log_param("best_catboost_iterations_H1", int(best_cat_iters_h1))

    # --- 3. Construção e Treinamento do Stacking ---

    # Define os estimadores finais com os parâmetros corretos
    lgbm_final = LGBMClassifier(**lgbm_params_2a)
    catboost_final = CatBoostClassifier(**{**cat_params_h1, "iterations": int(best_cat_iters_h1)})

    estimators = [
        ("lgbm", lgbm_final),
        ("catboost", catboost_final)
    ]

    # Define o meta-learner, conforme o script de referência
    final_estimator = RandomForestClassifier(
        n_estimators=100,
        max_depth=3,
        random_state=RANDOM_STATE,
        n_jobs=-1
    )

    # Cria e treina o StackingClassifier
    stacking_clf = StackingClassifier(
        estimators=estimators,
        final_estimator=final_estimator,
        cv=5,
        n_jobs=-1,
        passthrough=False,
        stack_method='predict_proba'
    )

    print("Iniciando o treinamento do StackingClassifier...")
    stacking_clf.fit(X_train, y_train)
    print("Treinamento concluído.")

    # --- 4. Avaliação Final ---

    print("Avaliando o modelo final...")
    evaluate_and_log_model(
        "stacking",
        "Stacking LGBM(2A) + CatBoost(H1 + ES) -> RF meta",
        stacking_clf,
        X_test, y_test
    )

print("\nProcesso finalizado.")

Iniciando o treinamento do StackingClassifier...


2025/09/15 16:29:26 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Treinamento concluído.
Avaliando o modelo final...


=== Avaliação do Modelo: Stacking LGBM(2A) + CatBoost(H1 + ES) -> RF meta ===
              precision    recall  f1-score   support

           0      0.763     0.819     0.790      8805
           1      0.820     0.774     0.796     15873
           2      0.723     0.758     0.740      5322

    accuracy                          0.784     30000
   macro avg      0.769     0.784     0.776     30000
weighted avg      0.786     0.784     0.785     30000

Recall da classe 0 (Poor): 0.819
Acurácia de Treino: 0.919
🏃 View run Stacking_LGBM_CatBoostH1_RFMeta at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/ca7b04fddf744f22861df1b200e05abf
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0

Processo finalizado.


### Conclusão (fechamento da fase atual)

Champion para seguir: Stacking com LGBM (2A) + CatBoost (3B1) + meta RF.

Por quê: melhorou Recall Poor (0,829 vs 0,818) e macro recall (0,788 vs 0,783) com pequeno custo em overfit (train 0,925 vs 0,919).

2A segue como baseline robusto (gap menor), útil para comparação.

3B2 foi descartado (queda forte no Recall Poor).

Sendo um modelo de stacking, sua complexidade é maior. No entanto, o ganho de performance justifica essa complexidade. A combinação de LightGBM, CatBoost e Random Forest como meta-learner parece ter criado um ensemble poderoso e generalista.
Em resumo, este modelo oferece a maior segurança de que tomará as decisões corretas na maior parte do tempo, minimizando tanto o risco de conceder crédito a maus pagadores quanto a perda de oportunidade ao negar crédito a bons pagadores.

## Otimizar XGBoost para empilhar

Objetivo: obter um XGB parcimonioso e complementar (árvores rasas, subsampling, L1/L2), não “bater” LGBM/CatBoost sozinho.

Método (Passo A): early stopping via CV para calibrar n_estimators.

Critérios de aceite (solo): Recall Poor ~0,78–0,80, treino ≤ 0,92, boa diversidade vs. LGBM/CatBoost.

Depois (Passo B opcional): random search pequeno (12×CV=3), refazendo ES no vencedor.

Empilhamento: incluir XGB como 3º base e reavaliar; depois calibração + thresholds.


In [ ]:
# === XGBoost 2-estágios (com MLflow): RS CV=3 → rechecagem top-3 CV=5 → xgb.cv p/ n_estimators → fit final ===
# Objetivo: obter XGB parcimonioso e diverso, pronto para entrar no stack. Mantém rastreamento no MLflow.



# ---------- Configs ----------
RANDOM_STATE = 42
N_CLASSES = 3
N_ITER_RS = 12          # RS enxuto (CV=3)
CV_RS = 3               # 1º estágio
CV_CHECK = 5            # rechecagem dos top-3
ES_ROUNDS = 40          # early stopping no xgb.cv
MAX_BOOST_ROUNDS = 1000

# Scorer principal
recall_macro_scorer = make_scorer(recall_score, average="macro", zero_division=0)

# Histórico do seu XGB anterior (para rastreio no MLflow)
prev_best = dict(max_depth=9, min_child_weight=3, subsample=1.0,
                 colsample_bytree=0.8, learning_rate=0.1, n_estimators=100)

# Estimador base (regularizado/diverso vs. anterior) — n_estimators moderado p/ RS
xgb_base_fixed = XGBClassifier(
    objective="multi:softprob",
    eval_metric="mlogloss",
    tree_method="hist",
    learning_rate=0.08,     # antigo: 0.10
    max_depth=6,            # antigo: 9
    min_child_weight=8,     # antigo: 3
    subsample=0.8,          # antigo: 1.0
    colsample_bytree=0.8,   # igual
    reg_lambda=2.0,         # L2
    reg_alpha=0.1,          # L1 leve
    gamma=0.05,             # ganho mínimo
    n_estimators=350,       # moderado; o ótimo virá do xgb.cv
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbosity=0
)

# Espaço de busca (pequeno e focado)
param_distributions = {
    "max_depth":         [4, 5, 6, 7],
    "min_child_weight":  [6, 8, 10, 12],
    "subsample":         [0.7, 0.8, 0.9],
    "colsample_bytree":  [0.7, 0.8, 0.9],
    "learning_rate":     [0.05, 0.08, 0.10],
    "reg_lambda":        [1.0, 2.0, 3.0, 5.0],
    "reg_alpha":         [0.0, 0.1, 0.2, 0.4],
    "gamma":             [0.0, 0.05, 0.1, 0.2],
}

with mlflow.start_run(run_name="XGB_2Stage_RS3_Top3CV5_CVbest"):
    # Metadados gerais
    mlflow.log_param("stage", "XGB_2Stage_RS3_TOP3CV5_CVbest")
    mlflow.log_param("rs_n_iter", N_ITER_RS)
    mlflow.log_param("rs_cv", CV_RS)
    mlflow.log_param("check_cv", CV_CHECK)
    mlflow.log_param("xgbcv_es_rounds", ES_ROUNDS)
    for k, v in prev_best.items():
        mlflow.log_param(f"xgb_prev_best_{k}", v)

    # ---------- Estágio 1: RandomizedSearchCV (CV=3) ----------
    skf_rs = StratifiedKFold(n_splits=CV_RS, shuffle=True, random_state=RANDOM_STATE)
    rs = RandomizedSearchCV(
        estimator=xgb_base_fixed,
        param_distributions=param_distributions,
        n_iter=N_ITER_RS,
        scoring=recall_macro_scorer,
        cv=skf_rs,
        random_state=RANDOM_STATE,
        n_jobs=-1,
        verbose=1,
        refit=True,                 # refit com n_estimators=350
        return_train_score=False
    )
    rs.fit(X_train, y_train)

    # Log básicos do RS
    mlflow.log_metric("rs_cv3_best_recall_macro", float(rs.best_score_))
    for k, v in rs.best_params_.items():
        mlflow.log_param(f"rs_cv3_best_{k}", v)

    # Artefatos: resultados completos do RS
    rs_df = pd.DataFrame(rs.cv_results_).sort_values("mean_test_score", ascending=False)
    rs_top3 = rs_df.head(3).copy()
    rs_all_path = "xgb_rs_cv3_results.csv"
    rs_top3_path = "xgb_rs_cv3_top3.csv"
    rs_df.to_csv(rs_all_path, index=False)
    rs_top3.to_csv(rs_top3_path, index=False)
    mlflow.log_artifact(rs_all_path, artifact_path="random_search")
    mlflow.log_artifact(rs_top3_path, artifact_path="random_search")

    # ---------- Estágio 2: Rechecagem dos top-3 com CV=5 ----------
    skf_check = StratifiedKFold(n_splits=CV_CHECK, shuffle=True, random_state=RANDOM_STATE)
    top3_params = [row["params"] for _, row in rs_top3.iterrows()]
    cv5_scores = []
    for i, p in enumerate(top3_params, start=1):
        cand = XGBClassifier(**xgb_base_fixed.get_params())
        cand.set_params(**p)
        score = cross_val_score(
            cand, X_train, y_train,
            cv=skf_check,
            scoring=recall_macro_scorer,
            n_jobs=-1
        ).mean()
        cv5_scores.append(score)
        mlflow.log_metric(f"top{i}_cv5_recall_macro", float(score))

    best_idx = int(np.argmax(cv5_scores))
    best_params_cv5 = top3_params[best_idx]
    mlflow.log_param("selected_from_top3_idx", best_idx + 1)
    for k, v in best_params_cv5.items():
        mlflow.log_param(f"cv5_selected_{k}", v)

    # ---------- Estágio 3: xgboost.cv para n_estimators ótimo ----------
    # Mapeia hiperparâmetros do melhor candidato para o formato do booster
    best_full = {**xgb_base_fixed.get_params(), **best_params_cv5}
    booster_params = {
        "objective": "multi:softprob",
        "num_class": N_CLASSES,
        "eval_metric": "mlogloss",
        "eta": best_full["learning_rate"],
        "max_depth": best_full["max_depth"],
        "min_child_weight": best_full["min_child_weight"],
        "subsample": best_full["subsample"],
        "colsample_bytree": best_full["colsample_bytree"],
        "lambda": best_full["reg_lambda"],
        "alpha": best_full["reg_alpha"],
        "gamma": best_full["gamma"],
        "tree_method": best_full.get("tree_method", "hist"),
        "verbosity": 0,
        "seed": RANDOM_STATE,
    }
    dtrain = xgb.DMatrix(X_train, label=y_train)
    cv_res = xgb.cv(
        params=booster_params,
        dtrain=dtrain,
        num_boost_round=MAX_BOOST_ROUNDS,
        nfold=CV_CHECK,
        early_stopping_rounds=ES_ROUNDS,
        stratified=True,
        verbose_eval=False,
        seed=RANDOM_STATE
    )
    best_rounds = int(cv_res["test-mlogloss-mean"].idxmin() + 1)
    mlflow.log_param("xgbcv_best_n_estimators", best_rounds)

    # ---------- Estágio 4: Fit final e avaliação ----------
    # Evitar duplicatas: use todos os params do best_full e só substitua n_estimators
    xgb_final_params = {**best_full, "n_estimators": int(best_rounds)}
    xgb_final = XGBClassifier(**xgb_final_params)
    xgb_final.fit(X_train, y_train, verbose=False)

    evaluate_and_log_model(
        kind="xgboost",
        model_name="XGBoost_2Stage_RS3_Top3CV5_CVbest",
        model=xgb_final,
        X_test=X_test,
        y_test=y_test
    )


2025/08/29 17:03:58 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Fitting 3 folds for each of 12 candidates, totalling 36 fits


2025/08/29 17:09:37 INFO mlflow.sklearn.utils: Logging the 5 best runs, 7 runs will be omitted.


=== Avaliação do Modelo: XGBoost_2Stage_RS3_Top3CV5_CVbest ===
              precision    recall  f1-score   support

           0      0.788     0.783     0.785      8805
           1      0.793     0.816     0.804     15873
           2      0.760     0.702     0.730      5322

    accuracy                          0.786     30000
   macro avg      0.780     0.767     0.773     30000
weighted avg      0.785     0.786     0.785     30000

Recall da classe 0 (Poor): 0.783
Acurácia de Treino: 0.959
🏃 View run XGB_2Stage_RS3_Top3CV5_CVbest at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/3284af96eb7e4d7d880d2a24bd3fef9f
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0



### XGBoost — Regra do 1-SE no xgb.cv e refit parcimonioso (para uso no stacking)

Aplicamos xgb.cv (cv=5, early stopping=40) para obter a curva de mlogloss e selecionar o n_estimators pelo critério 1-SE: escolhe-se o primeiro round cujo erro está dentro de (mínimo + desvio-padrão), priorizando generalização sobre ajuste fino.
O modelo usa objective="multi:softprob" (multi-classe), tree_method="hist" e os melhores hiperparâmetros atuais; apenas n_estimators é recalibrado pelo 1-SE.
Em seguida, fazemos o refit final com esse número de árvores mais parsimonioso, reduzindo a variância e o risco de overfitting sem alterar a arquitetura.
Todo o processo é rastreado no MLflow: parâmetros prévios, rounds mínimo e 1-SE, curva de CV (CSV/PNG) e métricas de avaliação no holdout.
A saída é um XGB mais enxuto e estável, pronto para entrar como base learner em um stacking (ou para comparação direta com outras variantes) mantendo foco em recall macro e classe Poor.

In [ ]:
# === XGBoost | Passo 1: aplicar regra do 1-SE no xgb.cv e refazer o fit final com menos árvores ===
# Objetivo: reduzir overfit escolhendo um n_estimators mais parcimonioso (≤ melhor round + 1 desvio-padrão).

RANDOM_STATE = 42
N_CLASSES = 3
ES_ROUNDS = 40
MAX_BOOST_ROUNDS = 1000

# Hiperparâmetros do melhor XGB atual (selecionados na etapa anterior)
best_full = dict(
    objective="multi:softprob",
    eval_metric="mlogloss",
    tree_method="hist",
    learning_rate=0.1,
    max_depth=7,
    min_child_weight=6,
    subsample=0.8,
    colsample_bytree=0.7,
    reg_lambda=5.0,
    reg_alpha=0.0,
    gamma=0.05,
    n_estimators=999,   # round mínimo anterior (cv); será substituído pelo 1-SE
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbosity=0
)

with mlflow.start_run(run_name="XGB_CV_1SE_Refit"):
    mlflow.log_param("stage", "XGB_CV_1SE_refit")
    for k, v in best_full.items():
        mlflow.log_param(f"xgb_prevsel_{k}", v)

    # 1) xgboost.cv para obter curva de mlogloss e aplicar 1-SE
    dtrain = xgb.DMatrix(X_train, label=y_train)
    booster_params = {
        "objective": best_full["objective"],
        "num_class": N_CLASSES,
        "eval_metric": best_full["eval_metric"],
        "eta": best_full["learning_rate"],
        "max_depth": best_full["max_depth"],
        "min_child_weight": best_full["min_child_weight"],
        "subsample": best_full["subsample"],
        "colsample_bytree": best_full["colsample_bytree"],
        "lambda": best_full["reg_lambda"],
        "alpha": best_full["reg_alpha"],
        "gamma": best_full["gamma"],
        "tree_method": best_full["tree_method"],
        "verbosity": 0,
        "seed": RANDOM_STATE,
    }
    cv_res = xgb.cv(
        params=booster_params,
        dtrain=dtrain,
        num_boost_round=MAX_BOOST_ROUNDS,
        nfold=5,
        early_stopping_rounds=ES_ROUNDS,
        stratified=True,
        verbose_eval=False,
        seed=RANDOM_STATE
    )

    # Índice (0-based) do menor logloss e valores
    min_idx = int(cv_res["test-mlogloss-mean"].idxmin())
    min_mean = float(cv_res["test-mlogloss-mean"].iloc[min_idx])
    min_std  = float(cv_res["test-mlogloss-std"].iloc[min_idx])
    threshold = min_mean + min_std

    # Regra 1-SE: primeiro round cujo mean ≤ threshold
    one_se_idx = int(np.where(cv_res["test-mlogloss-mean"].values <= threshold)[0][0])
    # Converter para 1-based (n_estimators)
    min_rounds   = min_idx + 1
    one_se_rounds = one_se_idx + 1

    mlflow.log_param("xgbcv_min_rounds",   min_rounds)
    mlflow.log_param("xgbcv_min_logloss",  round(min_mean, 6))
    mlflow.log_param("xgbcv_min_std",      round(min_std, 6))
    mlflow.log_param("xgbcv_1se_rounds",   one_se_rounds)
    mlflow.log_param("xgbcv_1se_threshold", round(threshold, 6))

    # Artefatos: curva do CV (csv + png)
    with tempfile.TemporaryDirectory() as tmpdir:
        curve_path = os.path.join(tmpdir, "xgb_cv_curve.csv")
        cv_res.to_csv(curve_path, index=True)
        mlflow.log_artifact(curve_path, artifact_path="xgb_cv")

        plt.figure(figsize=(7,4))
        plt.plot(cv_res["test-mlogloss-mean"].values, label="test-mlogloss-mean")
        plt.fill_between(
            range(len(cv_res)),
            cv_res["test-mlogloss-mean"].values - cv_res["test-mlogloss-std"].values,
            cv_res["test-mlogloss-mean"].values + cv_res["test-mlogloss-std"].values,
            alpha=0.2
        )
        plt.axvline(min_idx, color="tab:orange", linestyle="--", label=f"min @ {min_rounds}")
        plt.axvline(one_se_idx, color="tab:green", linestyle="--", label=f"1-SE @ {one_se_rounds}")
        plt.title("xgboost.cv — mlogloss (mean ± std)")
        plt.xlabel("round")
        plt.ylabel("mlogloss")
        plt.legend()
        png_path = os.path.join(tmpdir, "xgb_cv_curve.png")
        plt.tight_layout()
        plt.savefig(png_path, dpi=120)
        plt.close()
        mlflow.log_artifact(png_path, artifact_path="xgb_cv")

    # 2) Fit final com n_estimators = 1-SE (mais parcimonioso)
    xgb_final_params = {**best_full, "n_estimators": int(one_se_rounds)}
    xgb_final = XGBClassifier(**xgb_final_params)
    xgb_final.fit(X_train, y_train, verbose=False)

    # 3) Avaliação + logging usando sua função padrão
    evaluate_and_log_model(
        kind="xgboost",
        model_name="XGBoost_CV_1SE_Refit",
        model=xgb_final,
        X_test=X_test,
        y_test=y_test
    )

    # Prints úteis no console
    print(f"[xgb.cv] min_rounds={min_rounds}, min_logloss={min_mean:.6f}, std={min_std:.6f}")
    print(f"[xgb.cv] 1-SE rounds={one_se_rounds}, threshold={threshold:.6f} (usar este)")


=== Avaliação do Modelo: XGBoost_CV_1SE_Refit ===
              precision    recall  f1-score   support

           0      0.786     0.778     0.782      8805
           1      0.791     0.816     0.803     15873
           2      0.759     0.699     0.728      5322

    accuracy                          0.784     30000
   macro avg      0.778     0.764     0.771     30000
weighted avg      0.784     0.784     0.783     30000

Recall da classe 0 (Poor): 0.778
Acurácia de Treino: 0.944
[xgb.cv] min_rounds=999, min_logloss=0.550428, std=0.002408
[xgb.cv] 1-SE rounds=840, threshold=0.552835 (usar este)
🏃 View run XGB_CV_1SE_Refit at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/8fd73cf2e21047b1b39ce1092c9ad13c
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


Resultados com alto overfit 

## Xgboost como alvo ,nao para stack

###  XGBoost alvo — Rodada 1/2 (gbtree) com RandomizedSearchCV enxuto

Estratégia: tratar XGBoost como modelo-alvo (não para empilhar) e explorar um baseline gbtree com espaço pequeno porém “extremo” para captar direções claras.
Orçamento: 3 configs × CV=5 = 15 fits, priorizando recall_macro (negócio) e observando o recall da classe 0 (Poor).

In [ ]:
# === XGBoost alvo — Rodada 1/2 (gbtree apenas): RandomizedSearchCV enxuto (n_iter=3, CV=5) ===
# Objetivo: 3 configs × 5 folds = 15 fits. Espaço pequeno, porém “extremo”, só para gbtree.


with mlflow.start_run(run_name="XGB_Target_Round1_gbtree_RS3_CV5"):
    mlflow.log_param("stage", "XGB_target_round1_gbtree")
    mlflow.log_param("search_n_iter", 3)
    mlflow.log_param("search_cv", 5)

    xgb_base = XGBClassifier(
        objective="multi:softprob",
        num_class=3,
        tree_method="hist",
        eval_metric="mlogloss",
        booster="gbtree",         # sem DART nesta rodada
        grow_policy="depthwise",  # sem lossguide nesta rodada
        n_jobs=-1,
        random_state=42,
        verbosity=0,
    )

    # Espaço direcional (extremos úteis), sem DART/lossguide
    param_distributions = {
        "max_depth":        [5, 7],
        "min_child_weight": [8, 12],
        "gamma":            [0.0, 0.20],
        "reg_lambda":       [3.0, 8.0],
        "reg_alpha":        [0.0, 0.4],
        "subsample":        [0.65, 0.85],
        "colsample_bytree": [0.65, 0.85],
        "learning_rate":    [0.05, 0.08],
        "n_estimators":     [400, 800],
    }

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    search = RandomizedSearchCV(
        estimator=xgb_base,
        param_distributions=param_distributions,
        n_iter=3,                   # 15 fits no total
        scoring="recall_macro",
        cv=skf,
        n_jobs=-1,
        verbose=1,
        random_state=42,
        refit=True,                 # refita no treino completo com o melhor set
        return_train_score=False,
    )

    search.fit(X_train, y_train)

    # Logging do melhor resultado e grade completa
    mlflow.log_metric("best_cv_recall_macro", float(search.best_score_))
    for k, v in search.best_params_.items():
        mlflow.log_param(f"best_{k}", v)

    cv_df = pd.DataFrame(search.cv_results_).sort_values("mean_test_score", ascending=False)
    cv_df_path = "xgb_round1_gbtree_rs3_cv5_results.csv"
    cv_df.to_csv(cv_df_path, index=False)
    mlflow.log_artifact(cv_df_path, artifact_path="round1_results")

    # Avaliação final padronizada no teste
    best_model = search.best_estimator_
    evaluate_and_log_model(
        kind="xgboost",
        model_name="XGB_Target_Round1_gbtree_RS3_CV5_Best",
        model=best_model,
        X_test=X_test,
        y_test=y_test
    )


### XGBoost alvo — DART “single shot” (treino direto, sem CV/ES)

Estratégia rápida para medir ganho potencial do DART no recall (especialmente classe 0 – Poor) sem custo alto de busca.
Usa dropout de árvores (booster="dart", rate_drop, skip_drop, normalize_type="tree") para reduzir correlações entre árvores e variância.
Limitação consciente: sem CV/ES → resultado mais ruidoso; se promissor, evoluir para CV + early stopping antes de consolidar.


In [ ]:
# === XGBoost alvo — DART “single shot” (sem CV/ES): treino direto e avaliação ===
# Objetivo: testar rapidamente se DART entrega ganho em Recall da classe 0 sem custar tempo excessivo.


with mlflow.start_run(run_name="XGB_DART_SingleShot_Directional"):
    mlflow.log_param("stage", "XGB_dart_single_shot")

    # Configuração direcional (ajustes focados em generalização)
    dart_params = dict(
        objective="multi:softprob",
        num_class=3,
        eval_metric="mlogloss",
        tree_method="hist",     # Se estiver com GPU disponível, pode alternar para "gpu_hist"
        booster="dart",
        normalize_type="tree",
        rate_drop=0.15,
        skip_drop=0.50,
        # Capacidade/regularização
        max_depth=7,
        min_child_weight=10,
        gamma=0.10,
        reg_lambda=6.0,
        reg_alpha=0.3,
        # Amostragem
        subsample=0.75,
        colsample_bytree=0.75,
        # Regime de treino
        learning_rate=0.08,
        n_estimators=400,       # 350–500 sugerido; começamos com 400 para rapidez
        # Misc
        n_jobs=-1,
        random_state=42,
        verbosity=0,
    )

    # Log dos parâmetros para rastreabilidade
    mlflow.log_params({f"dart_{k}": v for k, v in dart_params.items()})

    # Treino direto (sem early stopping / sem CV)
    xgb_dart = XGBClassifier(**dart_params)
    xgb_dart.fit(X_train, y_train)

    # Avaliação padronizada + logging (inclui training_accuracy_score_manual e artefatos)
    evaluate_and_log_model(
        kind="xgboost",
        model_name="XGB_DART_SingleShot",
        model=xgb_dart,
        X_test=X_test,
        y_test=y_test
    )


### XGBoost alvo — lossGUIDE “probe” (RS n=1, CV=5) para direção de generalização

Exploramos grow_policy=lossguide (crescimento por nó com controle por max_leaves) como alternativa ao depthwise, visando reduzir variância mantendo capacidade de separar Poor.
Orçamento mínimo: 1 amostra × CV=5 = 5 fits (apenas um “ponto” na grade), suficiente para testar a direção sem custo alto; refit=True reentreina no treino completo.

In [ ]:
# === XGBoost alvo — Rodada 2 (parte 2): lossGUIDE-only (n_iter=1, CV=5) ===

with mlflow.start_run(run_name="XGB_Target_Round2_LOSSGUIDE_RS1_CV5"):
    mlflow.log_param("stage", "XGB_target_round2_lossguide_only")
    mlflow.log_param("search_n_iter", 1)
    mlflow.log_param("search_cv", 5)

    xgb_loss = XGBClassifier(
        objective="multi:softprob",
        num_class=3,
        tree_method="hist",
        eval_metric="mlogloss",
        booster="gbtree",
        grow_policy="lossguide",
        n_jobs=-1,
        random_state=42,
        verbosity=0,
    )

    param_distributions = {
        "n_estimators":     [400, 600],
        "learning_rate":    [0.05, 0.08],
        "max_leaves":       [32, 64],
        "max_depth":        [7, 8],
        "min_child_weight": [8, 12],
        "gamma":            [0.10, 0.20],
        "reg_lambda":       [5.0, 8.0],
        "reg_alpha":        [0.2, 0.4],
        "subsample":        [0.70, 0.85],
        "colsample_bytree": [0.70, 0.85],
    }

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    search = RandomizedSearchCV(
        estimator=xgb_loss,
        param_distributions=param_distributions,
        n_iter=1,                  # 1 x 5 = 5 fits
        scoring="recall_macro",
        cv=skf,
        n_jobs=-1,
        verbose=1,
        random_state=42,
        refit=True,
        return_train_score=False,
    )
    search.fit(X_train, y_train)

    mlflow.log_metric("lossguide_best_cv_recall_macro", float(search.best_score_))
    for k, v in search.best_params_.items():
        mlflow.log_param(f"lossguide_best_{k}", v)

    df = pd.DataFrame(search.cv_results_).sort_values("mean_test_score", ascending=False)
    csv_path = "round2_lossguide_rs_results.csv"
    df.to_csv(csv_path, index=False)
    mlflow.log_artifact(csv_path, artifact_path="round2_lossguide_only")

    best_model = search.best_estimator_
    evaluate_and_log_model(
        kind="xgboost",
        model_name="XGB_Target_Round2_LOSSGUIDE_RS1_CV5_Best",
        model=best_model,
        X_test=X_test,
        y_test=y_test
    )


### Avaliando Resultados

Conclusão  XGBoost como alvo/stack base

Resumo por variante

gbtree (Round1): Recall Poor 0,731, F1_macro 0,732, treino 0,838 → razoável, porém abaixo do necessário para o caso de uso.

lossguide: Recall Poor 0,712, F1_macro 0,721, treino 0,807 → intermediário; não supera gbtree.

DART: Recall Poor 0,700, F1_macro 0,708, treino 0,789 → mais fraco, tendência a subajuste.

Comparativo com baseline

Stack LGBM+CatBoost entrega Recall Poor ~0,818–0,821 com macro e accuracy superiores — margem clara sobre todos os XGB testados.

Leitura

Os XGB avaliados exibem viés alto na classe 0 e não atingem o patamar do stack atual.

Como base no stacking, só fariam sentido se agregassem diversidade útil (probabilidades pouco correlacionadas) sem degradar; com Recall Poor baixo, a chance de ganho é pequena.

Decisão prática

Depriorizar XGB como alvo e como base do stack neste momento.

## Catboost como alvo


### CatBoost alvo — C1: baseline anti-overfit com early stopping (eval_set) e autolog ON

Objetivo: testar o CatBoost como modelo-alvo com configuração parcimoniosa para conter overfitting, mantendo bom recall (especialmente classe 0 — Poor).
Técnica: split interno 90/10 do treino para eval_set e early stopping (od_type="Iter", od_wait=40, use_best_model=True), registrando a melhor iteração encontrada.
Regularização estrutural: depth=8 (árvores mais rasas) e l2_leaf_reg=8.0 (L2 ↑) para reduzir variância.
Bagging/robustez: bootstrap_type="Bayesian" com bagging_temperature=1.0 e rsm=0.8 (amostragem de colunas); random_strength=1.8 injeta ruído leve nos splits.
Regime de treino: iterations=1200, learning_rate=0.05; o ES escolhe o ponto ótimo antes do limite para evitar supertreino.

In [16]:
# === CatBoost alvo — C1: baseline anti-overfit (ES via eval_set), autolog ON ===


with mlflow.start_run(run_name="CatBoost_Target_C1_AntiOverfit_Baseline"):
    mlflow.log_param("stage", "CatBoost_C1_anti_overfit")

    # Split interno p/ early stopping (leve, estratificado)
    X_tr, X_val, y_tr, y_val = train_test_split(
        X_train, y_train, test_size=0.1, stratify=y_train, random_state=42
    )

    # Setup anti-overfit (capacidade ↓, regularização ↑, bagging/colsample)
    cb_params = dict(
        loss_function="MultiClass",
        eval_metric="TotalF1",
        iterations=1200,
        learning_rate=0.05,
        depth=8,                 # ↓ de 10 → 8
        l2_leaf_reg=8.0,         # ↑ de 3 → 8
        bootstrap_type="Bayesian",
        bagging_temperature=1.0,
        rsm=0.8,                 # column subsampling
        # subsample=0.8,         # (opcional; ignorado em Bayesian, manter comentado)
        random_strength=1.8,     # ruído leve p/ robustez
        od_type="Iter",
        od_wait=40,              # early stopping patience
        use_best_model=True,
        random_state=42,
        verbose=0
    )
    mlflow.log_params({f"C1_{k}": v for k, v in cb_params.items()})

    model = CatBoostClassifier(**cb_params)
    model.fit(X_tr, y_tr, eval_set=(X_val, y_val), verbose=0)

    # Log da melhor iteração encontrada pelo ES (quando disponível)
    try:
        best_iters = int(model.get_best_iteration())
    except Exception:
        best_iters = int(getattr(model, "tree_count_", 0) or 0)
    mlflow.log_param("C1_best_iterations", best_iters)

    # Avaliação padronizada (inclui training_accuracy_score_manual)
    evaluate_and_log_model(
        kind="catboost",
        model_name="CatBoost Target C1 AntiOverfit",
        model=model,
        X_test=X_test,
        y_test=y_test
    )


=== Avaliação do Modelo: CatBoost Target C1 AntiOverfit ===
              precision    recall  f1-score   support

           0      0.756     0.690     0.721      8805
           1      0.733     0.796     0.763     15873
           2      0.659     0.584     0.619      5322

    accuracy                          0.727     30000
   macro avg      0.716     0.690     0.701     30000
weighted avg      0.727     0.727     0.725     30000

Recall da classe 0 (Poor): 0.690
Acurácia de Treino: 0.772
🏃 View run CatBoost_Target_C1_AntiOverfit_Baseline at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/cb194197453c4127bd96ffe62f1d9480
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


### CatBoost alvo — C2: C1 + class_weights para puxar Recall da classe 0 (anti-overfit, autolog ON)

Extensão do C1 adicionando pesos de classe class_weights = [1.8, 1.0, 1.0] para favorecer a classe 0 (Poor) sem alterar a base anti-overfit.
Mantém regularização estrutural (depth=8, l2_leaf_reg=8.0), amostragem (bootstrap_type="Bayesian", bagging_temperature=1.0, rsm=0.8) e ruído controlado (random_strength=1.8).
Treino com ES em holdout interno 90/10 (od_type="Iter", od_wait=40, use_best_model=True) para parar antes do supertreino; iterations=1200 como teto.
Otimização por eval_metric="TotalF1"; avaliação padronizada no holdout final foca Recall_macro e Recall da classe 0.
MLflow registra estágio, hiperparâmetros efetivos, C2_best_iterations e métricas/artefatos para comparação direta C1 ↔ C2.
Expectativa: elevar Recall_Poor com potencial trade-off moderado em precisão das demais classes, sob controle do pacote anti-overfit.

In [17]:
# === CatBoost alvo — C2: C1 + class_weights para puxar Recall da classe 0, autolog ON ===

with mlflow.start_run(run_name="CatBoost_Target_C2_AntiOverfit_ClassWeights"):
    mlflow.log_param("stage", "CatBoost_C2_anti_overfit_class_weights")

    # Split interno p/ early stopping (mesmo do C1)
    X_tr, X_val, y_tr, y_val = train_test_split(
        X_train, y_train, test_size=0.1, stratify=y_train, random_state=42
    )

    # Mesmos hiperparâmetros do C1 + pesos de classe para favorecer a classe 0 (Poor)
    class_weights = [1.8, 1.0, 1.0]  # 0→1.8, 1→1.0, 2→1.0
    cb_params = dict(
        loss_function="MultiClass",
        eval_metric="TotalF1",
        iterations=1200,
        learning_rate=0.05,
        depth=8,
        l2_leaf_reg=8.0,
        bootstrap_type="Bayesian",
        bagging_temperature=1.0,
        rsm=0.8,
        random_strength=1.8,
        od_type="Iter",
        od_wait=40,
        use_best_model=True,
        class_weights=class_weights,
        random_state=42,
        verbose=0
    )
    mlflow.log_params({f"C2_{k}": v for k, v in cb_params.items() if k != "class_weights"})
    mlflow.log_param("C2_class_weights", "0:1.8,1:1.0,2:1.0")

    model = CatBoostClassifier(**cb_params)
    model.fit(X_tr, y_tr, eval_set=(X_val, y_val), verbose=0)

    # Log da melhor iteração do ES
    try:
        best_iters = int(model.get_best_iteration())
    except Exception:
        best_iters = int(getattr(model, "tree_count_", 0) or 0)
    mlflow.log_param("C2_best_iterations", best_iters)

    # Avaliação padronizada (inclui training_accuracy_score_manual)
    evaluate_and_log_model(
        kind="catboost",
        model_name="CatBoost Target C2 AntiOverfit + ClassWeights",
        model=model,
        X_test=X_test,
        y_test=y_test
    )


=== Avaliação do Modelo: CatBoost Target C2 AntiOverfit + ClassWeights ===
              precision    recall  f1-score   support

           0      0.702     0.812     0.753      8805
           1      0.773     0.738     0.755     15873
           2      0.682     0.598     0.637      5322

    accuracy                          0.735     30000
   macro avg      0.719     0.716     0.715     30000
weighted avg      0.736     0.735     0.734     30000

Recall da classe 0 (Poor): 0.812
Acurácia de Treino: 0.792
🏃 View run CatBoost_Target_C2_AntiOverfit_ClassWeights at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/fd6658dcc4c8448db17a0cbed5369aee
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


### CatBoost alvo — H1: busca leve ao redor do C2 (anti-overfit fino, RS=6, CV=5)

Explora um RandomizedSearchCV enxuto (n_iter=6, cv=5) em torno do C2 (com class_weights=[1.8,1.0,1.0]) para ajuste fino de regularização.
Treino com early stopping em eval_set fixo (split 90/10), teto iterations=1500 e od_wait=50 para parar antes do supertreino.
Espaço “direcional”: depth {7,8}, l2_leaf_reg {6,8,10,12}, rsm {0.7,0.8,0.9}, bagging_temperature {0.5,1.0,1.5,2.0}, random_strength {1.2,1.8,2.4}, border_count {32,64}, learning_rate {0.04,0.05}.
Configuração base: loss_function=MultiClass, eval_metric=TotalF1, bootstrap_type=Bayesian, rsm=0.8 (varrido), random_state=42, threads livres.
refit=True reaprende no treino completo com o melhor conjunto mantendo o mesmo eval_set para comparabilidade.

In [ ]:
# === CatBoost alvo — H1: busca leve ao redor do C2 (anti-overfit fino, CV=5, n_iter=6 ≈ 30 fits) ===

with mlflow.start_run(run_name="CatBoost_Target_H1_AroundC2_RS6_CV5"):
    mlflow.log_param("stage", "CatBoost_H1_around_C2")

    # Split interno fixo para ES (comparabilidade)
    X_tr, X_val, y_tr, y_val = train_test_split(
        X_train, y_train, test_size=0.10, stratify=y_train, random_state=42
    )

    # Base = C2 (classe 0 com peso 1.8) + anti-overfit
    cb_base = CatBoostClassifier(
        loss_function="MultiClass",
        eval_metric="TotalF1",
        iterations=1500,          # teto; ES escolhe melhor
        learning_rate=0.05,       # pode variar na busca
        depth=8,                  # pode variar (7,8)
        l2_leaf_reg=8.0,          # será varrido
        bootstrap_type="Bayesian",
        bagging_temperature=1.0,  # será varrido
        rsm=0.8,                  # será varrido
        random_strength=1.8,      # será varrido
        od_type="Iter",
        od_wait=50,               # ES um pouco mais paciente
        use_best_model=True,
        class_weights=[1.8, 1.0, 1.0],
        random_state=42,
        verbose=0,
        thread_count=-1,
    )

    # Espaço enxuto, “direcional”, ao redor do C2
    param_distributions = {
        "depth": [7, 8],
        "l2_leaf_reg": [6.0, 8.0, 10.0, 12.0],
        "rsm": [0.7, 0.8, 0.9],
        "bagging_temperature": [0.5, 1.0, 1.5, 2.0],
        "random_strength": [1.2, 1.8, 2.4],
        "border_count": [32, 64],
        "learning_rate": [0.04, 0.05],
    }

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    search = RandomizedSearchCV(
        estimator=cb_base,
        param_distributions=param_distributions,
        n_iter=6,                 # 6 x 5 = 30 fits
        scoring="recall_macro",
        cv=skf,
        n_jobs=-1,
        verbose=1,
        random_state=42,
        refit=True,               
        return_train_score=False,
    )

    # Passa eval_set para ES em cada fit (treino e refit)
    search.fit(X_tr, y_tr, eval_set=(X_val, y_val), verbose=False)

    # Log da busca
    mlflow.log_metric("H1_best_cv_recall_macro", float(search.best_score_))
    for k, v in search.best_params_.items():
        mlflow.log_param(f"H1_best_{k}", v)

    pd.DataFrame(search.cv_results_).sort_values("mean_test_score", ascending=False)\
        .to_csv("catboost_H1_rs6_cv5_results.csv", index=False)
    mlflow.log_artifact("catboost_H1_rs6_cv5_results.csv", artifact_path="H1_search")

    # Avaliação final no teste (usa seu helper, que também loga training_accuracy_score_manual)
    best_model = search.best_estimator_
    evaluate_and_log_model(
        kind="catboost",
        model_name="CatBoost_Target_H1_Best",
        model=best_model,
        X_test=X_test,
        y_test=y_test
    )


2025/09/03 13:16:48 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Fitting 5 folds for each of 6 candidates, totalling 30 fits


2025/09/03 13:50:38 INFO mlflow.sklearn.utils: Logging the 5 best runs, one run will be omitted.


🏃 View run intrigued-cub-238 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/f1f4a037b2ca4d7686922eb599df112b
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run zealous-stork-320 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/e365879efa0443b9b88db3f0b2bd70a7
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run unequaled-quail-6 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/0f867c24dfb14471864643d992ae9f22
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run omniscient-conch-778 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/5584fe845af44e9bbd141acaf62ae5b1
🧪 View experiment at: https://d

### CatBoost alvo — H2: ajuste fino do peso da classe 0 (Grid 3×, CV=5)

Extensão direta do H1: congelamos os hiperparâmetros vencedores e varremos apenas class_weights para priorizar a classe 0 (Poor) sem reabrir o espaço todo.
Configuração: GridSearchCV com 3 pesos ([1.6,1.0,1.2], [1.8,1.0,1.2], [1.6,1.0,1.4]) × CV=5 = 15 fits, scoring em recall_macro.
Treino usa early stopping em eval_set fixo (split 90/10) com od_wait=50, garantindo comparabilidade e corte de supertreino.
Base anti-overfit (de H1): depth=8, l2_leaf_reg=10, bootstrap_type=Bayesian, bagging_temperature=0.5, rsm=0.8, random_strength=1.8, lr=0.05.

In [16]:
# === CatBoost alvo — H2: ajuste fino do peso da classe 0 (Grid 3x, CV=5 = 15 fits) ===


with mlflow.start_run(run_name="CatBoost_Target_H2_ClassWeights_CV5-Correct"):
    mlflow.log_param("stage", "CatBoost_H2_class_weights")

    # Mesmo split para ES
    X_tr, X_val, y_tr, y_val = train_test_split(
        X_train, y_train, test_size=0.10, stratify=y_train, random_state=42
    )

    # Base: copie os melhores do H1 manualmente se quiser “congelar” tudo, exceto o peso.
    cb_base = CatBoostClassifier(
        loss_function="MultiClass",
        eval_metric="TotalF1",
        iterations=1500,
        learning_rate=0.05,   
        depth=8,              
        l2_leaf_reg=10.0,      
        bootstrap_type="Bayesian",
        bagging_temperature= 0.5,
        rsm=0.8,
        random_strength=1.8,
        od_type="Iter",
        od_wait=50,
        use_best_model=True,
        random_state=42,
        verbose=0,
        thread_count=-1,
    )

    # Varre apenas o peso da classe 0 (garante 3 x 5 = 15 fits)
    param_grid = {
        "class_weights": [
            [1.6, 1.0, 1.2],
            [1.8, 1.0, 1.2],
            [1.6, 1.0, 1.4],
        ]
    }

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    grid = GridSearchCV(
        estimator=cb_base,
        param_grid=param_grid,
        scoring="recall_macro",
        cv=skf,
        n_jobs=-1,
        verbose=1,
        refit=True,
        return_train_score=False,
    )

    grid.fit(X_tr, y_tr, eval_set=(X_val, y_val), verbose=False)

    # Log da busca
    mlflow.log_metric("H2_best_cv_recall_macro", float(grid.best_score_))
    mlflow.log_param("H2_best_class_weights", str(grid.best_params_["class_weights"]))

    pd.DataFrame(grid.cv_results_).sort_values("mean_test_score", ascending=False)\
        .to_csv("catboost_H2_classweights_cv5_results.csv", index=False)
    mlflow.log_artifact("catboost_H2_classweights_cv5_results.csv", artifact_path="H2_search")

    # Avaliação final
    best_model = grid.best_estimator_
    evaluate_and_log_model(
        kind="catboost",
        model_name="CatBoost_Target_H2_Best",
        model=best_model,
        X_test=X_test,
        y_test=y_test
    )


2025/09/15 10:59:50 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Fitting 5 folds for each of 3 candidates, totalling 15 fits


2025/09/15 11:23:53 INFO mlflow.sklearn.utils: Logging the 5 best runs, no runs will be omitted.


🏃 View run abundant-perch-722 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/b0281c6432134a278e808582bd79bb0e
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run invincible-jay-225 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/9dcc60945de340409023533c1a31f881
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


=== Avaliação do Modelo: CatBoost_Target_H2_Best ===
              precision    recall  f1-score   support

           0      0.724     0.800     0.760      8805
           1      0.803     0.725     0.762     15873
           2      0.642     0.718     0.678      5322

    accuracy                          0.746     30000
   macro avg      0.723     0.748     0.733     30000
weighted avg      0.751     0.746     0.747     30000

Recall da classe 0 (Poor): 0.800
Acurácia de Treino: 0.803
🏃 View run CatBoost_Target_H2_ClassWeights_CV5-Correct at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/5fc7f3ec254b47f19de8ce50241bf58c
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


🏃 View run selective-moose-347 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/05d33001cc05417595e82ad3975122a8
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


### CatBoost alvo — HL1: busca “heavy light” ao redor de H1/H2 (RS=15, CV=5) com foco em generalização

A cédula amplia levemente o espaço em torno do que funcionou em H1/H2 para subir recall_macro sem perder o ganho em Recall_Poor e sem aumentar a acurácia de treino.
Usamos RandomizedSearchCV (n_iter=15, cv=5) com early stopping em eval_set fixo (split 90/10) para encontrar um ponto estável de iterações (teto=1600, od_wait=50).
Mantemos a linha anti-overfit: depth∈{7,8}, l2_leaf_reg∈{8,10,12,14}, min_data_in_leaf∈{20,50,100}, Bayesian bootstrap com bagging_temperature∈{0.5,1.0,1.5}, rsm∈{0.70,0.80,0.90}.
Incluímos quantização (border_count∈{32,48,64}) e **lr∈{0.04,0.05}para modular capacidade, mantendoclass_weights=[1.6,1.0,1.4](H2). O *refit* usa o **mesmo eval_set** para comparabilidade;random_state=42garante reprodutibilidade. Todo o processo é **rastreado no MLflow**: melhores hiperparâmetros, tabela completa da busca e avaliação em *holdout* viaevaluate_and_log_model`.

In [20]:
# === CatBoost alvo — HL1: busca “heavy light” ao redor do H1/H2 (CV=5, n_iter=15 ≈ 75 fits) ===
# Foco: ↑ recall_macro SEM perder o ganho de Recall Poor e SEM subir a acurácia de treino.


with mlflow.start_run(run_name="CatBoost_Target_HL1_RS15_CV5"):
    mlflow.log_param("stage", "CatBoost_HL1_heavy_light")
    mlflow.log_param("hl1_n_iter", 15)
    mlflow.log_param("hl1_cv_splits", 5)

    # Split interno fixo para ES (comparável a H1/H2)
    X_tr, X_val, y_tr, y_val = train_test_split(
        X_train, y_train, test_size=0.10, stratify=y_train, random_state=42
    )

    # Base (partindo do que funcionou em H1/H2) + ES
    cb_base = CatBoostClassifier(
        loss_function="MultiClass",
        eval_metric="TotalF1",
        iterations=1600,          # teto; ES escolhe o melhor
        learning_rate=0.05,       # pode variar no espaço
        depth=8,                  # pode variar (7, 8)
        l2_leaf_reg=10.0,         # será varrido
        bootstrap_type="Bayesian",
        bagging_temperature=0.5,  # será varrido
        rsm=0.8,                  # será varrido
        random_strength=1.8,      # será varrido
        border_count=32,          # será varrido
        # regularização estrutural adicional
        # (varremos min_data_in_leaf no espaço abaixo)
        od_type="Iter",
        od_wait=50,               # ES um pouco mais paciente
        use_best_model=True,
        class_weights=[1.6, 1.0, 1.4],  # melhor do H2
        random_state=42,
        verbose=0
    )

    # Espaço “direcional” (enxuto, porém mais amplo que H1)
    param_distributions = {
        # capacidade / regularização
        "depth": [7, 8],
        "l2_leaf_reg": [8.0, 10.0, 12.0, 14.0],
        "min_data_in_leaf": [20, 50, 100],
        "random_strength": [1.2, 1.8, 2.4],
        # amostragem / colunas
        "bagging_temperature": [0.5, 1.0, 1.5],
        "rsm": [0.70, 0.80, 0.90],
        # quantização e passo
        "border_count": [32, 48, 64],
        "learning_rate": [0.04, 0.05],
    }

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    search = RandomizedSearchCV(
        estimator=cb_base,
        param_distributions=param_distributions,
        n_iter=15,                 # 15 x 5 = ~75 fits
        scoring="recall_macro",
        cv=skf,
        n_jobs=-1,
        verbose=1,
        random_state=42,
        refit=True,                # refita com o melhor set (com mesmo eval_set)
        return_train_score=False,
    )

    # Muito importante: passar eval_set para que o ES funcione nos folds e no refit
    search.fit(X_tr, y_tr, eval_set=(X_val, y_val), verbose=False)

    # Logging do melhor resultado e da tabela completa
    mlflow.log_metric("HL1_best_cv_recall_macro", float(search.best_score_))
    for k, v in search.best_params_.items():
        mlflow.log_param(f"HL1_best_{k}", v)

    cv_df = pd.DataFrame(search.cv_results_).sort_values("mean_test_score", ascending=False)
    csv_path = "catboost_HL1_rs15_cv5_results.csv"
    cv_df.to_csv(csv_path, index=False)
    mlflow.log_artifact(csv_path, artifact_path="HL1_search")

    # Avaliação final no teste com seu helper (inclui training_accuracy_score_manual + artefatos)
    best_model = search.best_estimator_
    evaluate_and_log_model(
        kind="catboost",
        model_name="CatBoost_Target_HL1_Best",
        model=best_model,
        X_test=X_test,
        y_test=y_test
    )


2025/09/03 15:51:56 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Fitting 5 folds for each of 15 candidates, totalling 75 fits


2025/09/03 17:24:05 INFO mlflow.sklearn.utils: Logging the 5 best runs, 10 runs will be omitted.


=== Avaliação do Modelo: CatBoost_Target_HL1_Best ===
              precision    recall  f1-score   support

           0      0.713     0.790     0.749      8805
           1      0.795     0.713     0.752     15873
           2      0.625     0.706     0.663      5322

    accuracy                          0.734     30000
   macro avg      0.711     0.736     0.721     30000
weighted avg      0.741     0.734     0.735     30000

Recall da classe 0 (Poor): 0.790
Acurácia de Treino: 0.786
🏃 View run CatBoost_Target_HL1_RS15_CV5 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/191a454cecf94841904f13b651897c35
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


##

##  Seleção do Modelo Final para Produção

### 1. Modelo Selecionado

Após uma série de experimentos rigorosos, incluindo a otimização de modelos base e a avaliação de diferentes arquiteturas de ensemble, o modelo selecionado para ser o candidato principal à produção é:

**`Stacking LGBM(2A) + CatBoost(3B1 Bernoulli + ES) → RF meta`**

Este modelo demonstrou a melhor combinação de performance, robustez e alinhamento com os objetivos de negócio, superando os demais candidatos em métricas críticas.

### 2. Análise de Performance e Justificativa Técnica

A superioridade deste modelo não se baseia em uma única métrica, mas em uma análise holística de seu comportamento, especialmente em comparação com seu concorrente mais próximo (a versão com CatBoost H1).

#### Prós e Vantagens Competitivas:

1.  **Performance Superior nas Métricas de Negócio:**
    *   **Recall da Classe "Poor" (Risco):** Com um **Recall de 0.829** para a classe 0, este modelo é o mais eficaz em identificar clientes de alto risco, cumprindo o principal objetivo de um sistema de prevenção de perdas.
    *   **Recall Macro (Equilíbrio):** Atingiu o maior **Recall Macro (0.788)**, indicando que ele possui o melhor equilíbrio na identificação correta de *todas* as classes (Poor, Standard e Good). Isso significa que, além de mitigar o risco, ele também minimiza a perda de oportunidades ao não negar crédito indevidamente a bons clientes.
    *   **Recall da Classe "Good" (Oportunidade):** Com **0.772**, também se mostrou o mais competente em reconhecer os clientes de baixo risco, reforçando sua capacidade de segmentação em todo o espectro de clientes.

2.  **Arquitetura de Ensemble Robusta:**
    *   **Diversidade dos Modelos Base:** O modelo combina a força do **LightGBM** (com seu crescimento de árvore *leaf-wise*) e do **CatBoost** (com suas árvores simétricas e tratamento de categóricas). A versão "3B1" do CatBoost, que utiliza `bootstrap_type='Bernoulli'` e `subsample`, introduziu uma camada adicional de regularização e amostragem, criando um modelo base mais generalista e, paradoxalmente, mais poderoso dentro do ensemble.
    *   **Meta-Learner Simples e Eficaz:** O uso de um `RandomForestClassifier` de baixa complexidade (`max_depth=3`) como meta-learner provou ser extremamente eficaz. Ele aprende a combinar as previsões dos modelos base de forma não-linear, mas sem a complexidade excessiva que poderia levar a um overfitting na segunda camada do stacking.

#### Análise do Overfitting: A Diferença entre Treino e Teste

Observou-se uma diferença entre a performance de treino e a de teste:

*   **Acurácia de Treino:** 0.925
*   **Acurácia de Teste:** 0.784

Embora a diferença de ~14 pontos percentuais possa parecer um sinal de alerta para overfitting, ela deve ser interpretada dentro do contexto de modelos de stacking de alta performance.

**Argumentação sobre a Qualidade do Modelo:**

1.  **Overfitting Esperado e Controlado:** Modelos de Gradient Boosting, e especialmente ensembles de stacking, são projetados para ter uma capacidade de aprendizado extremamente alta. É natural e esperado que eles se ajustem quase perfeitamente aos dados de treino. O verdadeiro teste de um modelo não é a ausência de overfitting, mas sua **capacidade de generalização**, ou seja, sua performance em dados nunca vistos (o conjunto de teste).

2.  **A Generalização é o que Importa:** O que define a qualidade do modelo para o negócio é sua performance no teste. Com um **Recall Macro de 0.788** e um **Recall "Poor" de 0.829**, este modelo prova que, apesar de ter aprendido os dados de treino em detalhes, ele conseguiu extrair padrões que generalizam de forma eficaz e entregam um valor comercial tangível e superior aos outros candidatos.

3.  **Complexidade que se Traduz em Performance:** A maior acurácia de treino do modelo vencedor (0.925 vs. 0.919 do concorrente) indica que a configuração do CatBoost "3B1" permitiu um aprendizado mais profundo. O ponto crucial é que essa capacidade de aprendizado adicional **se traduziu diretamente em melhores métricas de teste**, validando que o modelo não estava apenas memorizando ruído, mas sim capturando uma representação mais rica e útil dos dados.

### 3. Conclusão

O modelo `Stacking LGBM(2A) + CatBoost(3B1)` representa o ápice de nossa experimentação. Ele oferece o melhor equilíbrio entre a mitigação de risco (maior Recall "Poor") e a maximização de oportunidades (maior Recall "Good" e "Macro"), tornando-o a escolha técnica mais sólida e segura para avançar para as próximas etapas de validação e implantação em produção.


### Validação cruzada externa do ensemble vencedor (Stacking LGBM(2A) + CatBoost(3B1) → RF meta)

Executamos uma CV externa estratificada k=5 para estimar generalização e variância por fold, evitando viés do holdout único. Em cada fold externo, reajustamos apenas iterations do CatBoost via um early stopping leve (paciente=45) usando um único split interno do treino do próprio fold, reduzindo custo sem vazamento. O LGBM fica fixo com os hiperparâmetros do passo 2A, e treinamos um StackingClassifier com stack_method='predict_proba' e RF raso como meta-learner. Medimos em cada validação externa recall_macro, recall da classe 0 (Poor) e f1_macro, agregando média ± desvio-padrão para robustez. Registramos no MLflow: parâmetros do pipeline, métricas agregadas, e CSV por fold (rastreabilidade). Objetivo: confirmar estabilidade do ganho em Poor e detectar overfitting (ex.: training drift) antes de promover o modelo.

In [27]:
# ==============================================================================
# === HELPER: estima 'iterations' do CatBoost com ES em um fold interno rápido ==
# ==============================================================================

def _best_iters_catboost(X, y, base_params, n_splits=5, patience=45, random_state=42):
    """Estima 'iterations' para CatBoost via uma CV interna simples (rápida).
    Usa apenas o 1º fold para não aninhar CVs completas.
    """
    skf_inner = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    tr_idx, va_idx = next(iter(skf_inner.split(X, y)))
    X_tr, X_va = (X.iloc[tr_idx], X.iloc[va_idx]) if hasattr(X, "iloc") else (X[tr_idx], X[va_idx])
    y_tr, y_va = (y.iloc[tr_idx], y.iloc[va_idx]) if hasattr(y, "iloc") else (y[tr_idx], y[va_idx])

    cb = CatBoostClassifier(**base_params)
    cb.fit(X_tr, y_tr, eval_set=(X_va, y_va), verbose=False,
           early_stopping_rounds=patience, use_best_model=True)
    it = cb.get_best_iteration()
    return int(it if it is not None and it > 0 else base_params.get("iterations", 1200))


# ==============================================================================
# === PIPELINE DE VALIDAÇÃO CRUZADA EXTERNA ====================================
# ==============================================================================

RUN_NAME = "ExternalCV_Stacking_LGBM_CatBoost3B1_RFMeta"
RANDOM_STATE = 42
N_SPLITS_EXTERNO = 5  # folds da validação externa

# 1) Parâmetros base (modelo ganhador)
lgbm_params_2a = {
    'learning_rate': 0.1, 'colsample_bytree': 0.8, 'min_split_gain': 0.1,
    'subsample': 0.7, 'subsample_freq': 1, 'num_leaves': 95, 'max_depth': 10,
    'min_child_samples': 60, 'reg_lambda': 1.0, 'reg_alpha': 0.1,
    'objective': "multiclass", 'num_class': 3, 'random_state': RANDOM_STATE,
    'n_estimators': 408
}
cat_base_params_3b1 = {
    'learning_rate': 0.05, 'depth': 7, 'l2_leaf_reg': 12, 'rsm': 0.80,
    'bootstrap_type': "Bernoulli", 'subsample': 0.75, 'random_strength': 2.5,
    'border_count': 48, 'loss_function': "MultiClass", 'eval_metric': "TotalF1",
    'verbose': 0, 'random_state': RANDOM_STATE
}

# 2) Desliga autolog durante a CV externa (evita runs/fits internos no MLflow)
try:
    mlflow.autolog(disable=True)
except Exception:
    pass

skf_externo = StratifiedKFold(n_splits=N_SPLITS_EXTERNO, shuffle=True, random_state=RANDOM_STATE)
fold_metrics = []

print(f"Iniciando Validação Cruzada Externa com {N_SPLITS_EXTERNO} folds...")

for fold_num, (train_idx, val_idx) in enumerate(skf_externo.split(X, y), 1):
    print(f"\n--- Processando Fold Externo {fold_num}/{N_SPLITS_EXTERNO} ---")

    # Split do fold
    X_train_fold = X.iloc[train_idx] if hasattr(X, "iloc") else X[train_idx]
    y_train_fold = y.iloc[train_idx] if hasattr(y, "iloc") else y[train_idx]
    X_val_fold   = X.iloc[val_idx]   if hasattr(X, "iloc") else X[val_idx]
    y_val_fold   = y.iloc[val_idx]   if hasattr(y, "iloc") else y[val_idx]

    # CatBoost: estima iterations com ES usando apenas dados de treino do fold
    cat_params_for_es = {**cat_base_params_3b1, 'iterations': 2000}
    best_cat_iters = _best_iters_catboost(
        X_train_fold, y_train_fold,
        base_params=cat_params_for_es,
        n_splits=3, patience=45, random_state=RANDOM_STATE
    )
    print(f"Fold {fold_num}: Iterações ótimas para CatBoost estimadas em {best_cat_iters}")

    # Estimadores finais do fold
    lgbm_final = LGBMClassifier(**lgbm_params_2a)
    catboost_final = CatBoostClassifier(**{**cat_base_params_3b1, "iterations": best_cat_iters})
    estimators = [("lgbm", lgbm_final), ("catboost", catboost_final)]

    # CV do Stacking: usar StratifiedKFold embaralhada para reprodutibilidade
    skf_stack = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

    final_estimator = RandomForestClassifier(
        n_estimators=100, max_depth=3, random_state=RANDOM_STATE, n_jobs=-1
    )
    stacking_clf_fold = StackingClassifier(
        estimators=estimators,
        final_estimator=final_estimator,
        cv=skf_stack,              # <= ajuste 1: CV estratificada e embaralhada
        n_jobs=-1,
        passthrough=False,
        stack_method='predict_proba'
    )

    print(f"Fold {fold_num}: Treinando o StackingClassifier...")
    stacking_clf_fold.fit(X_train_fold, y_train_fold)

    # Predição no holdout do fold
    y_pred_fold = stacking_clf_fold.predict(X_val_fold)

    # Métricas por fold
    per_class_recall = recall_score(
        y_val_fold, y_pred_fold, labels=[0, 1, 2], average=None, zero_division=0
    )
    recall_poor = float(per_class_recall[0])  # <= ajuste 2: recall explícito da classe 0
    recall_macro = float(recall_score(y_val_fold, y_pred_fold, average='macro', zero_division=0))
    f1_macro = float(f1_score(y_val_fold, y_pred_fold, average='macro', zero_division=0))

    fold_metrics.append({
        "fold": fold_num,
        "recall_macro": recall_macro,
        "recall_poor": recall_poor,
        "f1_macro": f1_macro
    })
    print(f"Fold {fold_num}: Recall Macro = {recall_macro:.4f}, Recall Poor = {recall_poor:.4f}")

# 3) Agregação e logging único no MLflow
metrics_df = pd.DataFrame(fold_metrics)

print("\n\n" + "="*60)
print("=== RESULTADO DA VALIDAÇÃO CRUZADA EXTERNA ===")
print("="*60)
print(metrics_df.round(4).to_string(index=False))

print("\n--- Estatísticas Agregadas ---")
mean_metrics = metrics_df.mean(numeric_only=True)
std_metrics = metrics_df.std(numeric_only=True)
print(f"Recall Macro: {mean_metrics['recall_macro']:.4f} ± {std_metrics['recall_macro']:.4f}")
print(f"Recall Poor:  {mean_metrics['recall_poor']:.4f} ± {std_metrics['recall_poor']:.4f}")
print(f"F1 Macro:     {mean_metrics['f1_macro']:.4f} ± {std_metrics['f1_macro']:.4f}")
print("="*60)

with mlflow.start_run(run_name=RUN_NAME):
    mlflow.log_param("model_name", "Stacking_LGBM_CatBoost3B1_RFMeta")
    mlflow.log_param("external_cv_folds", N_SPLITS_EXTERNO)

    mlflow.log_metric("mean_recall_macro", float(mean_metrics['recall_macro']))
    mlflow.log_metric("std_recall_macro", float(std_metrics['recall_macro']))
    mlflow.log_metric("mean_recall_poor", float(mean_metrics['recall_poor']))
    mlflow.log_metric("std_recall_poor", float(std_metrics['recall_poor']))
    mlflow.log_metric("mean_f1_macro", float(mean_metrics['f1_macro']))
    mlflow.log_metric("std_f1_macro", float(std_metrics['f1_macro']))

    metrics_df.to_csv("external_cv_results.csv", index=False)
    mlflow.log_artifact("external_cv_results.csv")

print("\nProcesso de validação externa concluído e resultados logados no MLflow.")

# 4) Reativa autolog para os próximos experimentos
try:
    mlflow.autolog(disable=False)
except Exception:
    pass


Iniciando Validação Cruzada Externa com 5 folds...



--- Processando Fold Externo 1/5 ---
Fold 1: Iterações ótimas para CatBoost estimadas em 1130
Fold 1: Treinando o StackingClassifier...
Fold 1: Recall Macro = 0.7993, Recall Poor = 0.8520

--- Processando Fold Externo 2/5 ---
Fold 2: Iterações ótimas para CatBoost estimadas em 1997
Fold 2: Treinando o StackingClassifier...
Fold 2: Recall Macro = 0.7955, Recall Poor = 0.8343

--- Processando Fold Externo 3/5 ---
Fold 3: Iterações ótimas para CatBoost estimadas em 1738
Fold 3: Treinando o StackingClassifier...
Fold 3: Recall Macro = 0.7936, Recall Poor = 0.8371

--- Processando Fold Externo 4/5 ---
Fold 4: Iterações ótimas para CatBoost estimadas em 761
Fold 4: Treinando o StackingClassifier...
Fold 4: Recall Macro = 0.8013, Recall Poor = 0.8540

--- Processando Fold Externo 5/5 ---
Fold 5: Iterações ótimas para CatBoost estimadas em 1571
Fold 5: Treinando o StackingClassifier...
Fold 5: Recall Macro = 0.7899, Recall Poor = 0.8378


=== RESULTADO DA VALIDAÇÃO CRUZADA EXTERNA ===
 fold  

🏃 View run ExternalCV_Stacking_LGBM_CatBoost3B1_RFMeta at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/5325f20a2ed24546bfcd60ae7eee7673
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0

Processo de validação externa concluído e resultados logados no MLflow.


2025/09/15 18:32:37 INFO mlflow.tracking.fluent: Autologging successfully enabled for lightgbm.
2025/09/15 18:32:37 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/09/15 18:32:38 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.


### Validação cruzada externa — confirmação dos resultados (k=5)

Ensemble avaliado: Stacking LGBM(2A) + CatBoost(3B1) → RF meta, com recalibração de iterations do CatBoost por fold via ES. Os números ficaram coerentes e estáveis entre os folds, indicando boa generalização e baixo risco de overfit adicional.
Métricas agregadas (média ± desvio): Recall Macro 0,7959 ± 0,0045, Recall Poor 0,8430 ± 0,0092, F1 Macro 0,7836 ± 0,0034.
Por fold, o Recall Poor ficou de 0,8343 a 0,8540, mantendo-se acima do patamar-alvo; o Recall Macro variou pouco (0,7899–0,8013).
Embora as iterações ótimas do CatBoost tenham variado (761–1997), a performance permaneceu estável — bom sinal de robustez do ensemble.

# Registro de Modelo em Produção

Após treinar e avaliar o modelo, podemos registrá-lo oficialmente no **Model Registry do MLflow**.  
Isso permite versionar o modelo, promovê-lo para produção e gerenciar futuras atualizações.  

- Usamos o **run_id** obtido no link do MLflow (na interface Dagshub).
- Escolhemos um nome amigável e consistente para o modelo, neste caso: `credit-score-model`.

In [28]:
run_id = "bccfb26c333a41cf94facfc225cc8f2c"

mlflow.register_model(
    model_uri=f"runs:/{run_id}/model",
    name="credit-score-model"
)

print(f" Modelo registrado como 'credit-score-model' (run_id: {run_id})")

Registered model 'credit-score-model' already exists. Creating a new version of this model...
2025/09/17 12:16:42 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: credit-score-model, version 3
Created version '3' of model 'credit-score-model'.


 Modelo registrado como 'credit-score-model' (run_id: bccfb26c333a41cf94facfc225cc8f2c)
